#### Update of this notebook:
- *IRS Form 990 e-File Data (5) -- Parse Nested 'Sub-Key' Variables -- DOWNLOADED XML FILINGS.ipynb*

# Overview

This is the fifth in a series of tutorials that illustrate how to download, extract, and parse the IRS 990 e-file data available at https://aws.amazon.com/public-data-sets/irs-990/

In the previous notebook we used the information contained in the concordance file to combine pairs of columns that reflect the same 990 variable, such as *TaxPeriodBeginDt* and *TaxPeriodBeginDate*, and assign the relevant 'standardized' name from the concordance file, and then 'binarized' relevant columns and, lastly, deleted unneeded columns.


The goal of this notebook is to parse all of the 'dictionary' columns, or those with 'nested' dictionary structures. For example, the data for one observation in the column in the XML file called *Filer* might look like this:

``{'EIN': '203840246', 'Name': {'BusinessNameLine1': 'NEW ALBANY WALKING CLUB INC'}, 'NameControl': 'NEWA', 'USAddress': {'AddressLine1': '4000 BAUGHMAN GRANT', 'City': 'NEW ALBANY', 'State': 'OH', 'ZIPCode': '43054'}}``

And the data for *F9_10_PC_UNSECURED_NOTES_BOY* may look like this:

``{'BOYAmt': '24000', 'EOYAmt': '47479'}``

In effect, multiple variables are nested under the same column extracted from the raw e-file data. In the concordance file I have added a column, called 'sub_key', that tells us that, for the variable *F9_10_PC_UNSECURED_NOTES_BOY*, we will want to extract data nested under the ``EOYAmt`` and ``EOY`` keys. I added this data after conducting extensive verifications on the data. 

Accordingly, our first step  in this notebook will be to read in the concordance file that has all the reconciled and verified variables to date:
- The file is called *concordance_VERIFIED.xlsx*

We then read in the PANDAS data file (N=2,104,435) saved in our last notebook: 
- *all filings August 2022 - all control variables (renamed).pkl.gz*

I then parse all columns that have Python dictionaries as values and then save an updated e-file dataframe:
- *all filings August 2022 - all control variables (with parsed sub-key variables).pkl.gz*

# Load Packages and Connect to MongoDB

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
print(pd.__version__)

2.2.2


In [3]:
#http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 250)

#### Set working directory

In [4]:
cd "C:\\Users\\Gregory\\IRS 990 Control Variables\\"

C:\Users\Gregory\IRS 990 Control Variables


# Read in Concordance File
Read in the 'concordance' file. This codebook will help us identify the variables that contain dictionaries. We will then use the 'sub-key' column to help parse these columns. 

In [5]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
concordance = pd.read_excel('concordance_VERIFIED.xlsx')
print('# of columns:', len(concordance.columns))
print('# of observations:', len(concordance))
concordance[:2]

Current date and time :  2025-04-17 16:11:58 

# of columns: 17
# of observations: 574
CPU times: total: 297 ms
Wall time: 6.7 s


,xpath,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,python_data_type,fill_null,BINARIZE,MongoDB_Name,sub_key,sub_sub_key,cardinality
0,/Return/ReturnData/IRS990/SpecialConditionDesc,F9_00_HD_SPECIAL_CONDITION_DESC,NaN,NaN,NaN,NaN,Special condition description,F990-PC-PART-00,PART-00,TextType,string,Do not fill null,NaN,SpecialConditionDesc,NaN,NaN,NaN
1,/Return/ReturnData/IRS990/SpecialConditionDescription,F9_00_HD_SPECIAL_CONDITION_DESC,31.0,NaN,NaN,NaN,Special condition description,F990-PC-PART-00,PART-00,TextType,string,Do not fill null,NaN,SpecialConditionDescription,NaN,NaN,NaN


In [6]:
concordance['cardinality'].value_counts()

cardinality
ONE     88
MANY     2
Name: count, dtype: int64

In [7]:
concordance[concordance['cardinality']=='MANY']

,xpath,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,python_data_type,fill_null,BINARIZE,MongoDB_Name,sub_key,sub_sub_key,cardinality
422,/Return/ReturnData/IRS990/OtherExpenses/Total,F9_09_EXP_OTH_TOT,NaN,NaN,NaN,NaN,All Other expenses - total expense,F990-PC-PART-09-LINE-24A,PART-09,USAmountType,Int64,NaN,NaN,OtherExpenses,Total,NaN,MANY
423,/Return/ReturnData/IRS990/OtherExpensesGrp/TotalAmt,F9_09_EXP_OTH_TOT,NaN,NaN,NaN,NaN,Other Expenses - total expense,F990-PC-PART-09-LINE-24A,PART-09,USAmountType,Int64,NaN,NaN,OtherExpensesGrp,TotalAmt,NaN,MANY


In [8]:
concordance[concordance['sub_key'].notnull()][['variable_name_new', 'MongoDB_Name', 'sub_key']]

,variable_name_new,MongoDB_Name,sub_key
122,F9_03_PC_PROG_SVC_ACC_2_CODE,Activity2,ActivityCode
123,F9_03_PC_PROG_SVC_ACC_2_CODE,ProgSrvcAccomActy2Grp,ActivityCode
124,F9_03_PC_PROG_SVC_ACC_3_CODE,Activity3,ActivityCode
125,F9_03_PC_PROG_SVC_ACC_3_CODE,ProgSrvcAccomActy3Grp,ActivityCode
146,F9_03_PC_PROG_SVC_ACC_2_DESC,Activity2,Description
...,...,...,...
556,F9_00_HD_FILER_CITY_US,Filer,USAddress
557,F9_00_HD_FILER_COUNTRY_FRGN,Filer,ForeignAddress
558,F9_00_HD_FILER_COUNTRY_FRGN,Filer,ForeignAddress
559,F9_00_HD_FILER_ZIP_US,Filer,USAddress


In [9]:
subkeycols = list(set(concordance[concordance['sub_key'].notnull()]['variable_name_new'].tolist()))
print(len(subkeycols))
subkeycols

109


['F9_09_PC_TOTAL_FUNC_EXPENSES',
 'F9_10_PC_SAVINGS_TEMP_INVEST_EOY',
 'F9_09_PC_COMP_DISQUAL_MGMT',
 'F9_00_HD_FILER_COUNTRY_FRGN',
 'F9_09_PC_TOTAL_PROG_SVCE_EXPENSE',
 'F9_09_PC_PAYROLL_TAX_FUNDRAISE',
 'F9_10_PC_SECURE_MORT_NOTES_EOY',
 'F9_09_PC_OTHER_SALARY_TOTAL',
 'F9_09_PC_PAYROLL_TAX_PROG_SVCE',
 'F9_10_PC_CASH_NON_INTEREST_BOY',
 'F9_09_EXP_OFFICE_TOT',
 'F9_10_ASSETS_INTANGIB_EOY',
 'F9_10_LIAB_GRANTS_PAYABLE_EOY',
 'F9_09_PC_COMP_OFFICERS_MGMT',
 'F9_10_ASSETS_OTH_EOY',
 'F9_09_EXP_OTH_OTH_TOT',
 'F9_09_PC_OTHER_EMP_BEN_MGMT',
 'F9_09_EXP_CONF_MEETING_TOT',
 'F9_09_PC_FEES_FOR_SVCE_LOBB_TOT',
 'F9_00_HD_SIGNING_OFFICER_SIGNTR',
 'F9_09_EXP_JOINT_COSTS_TOT',
 'F9_09_EXP_AD_PROMO_TOT',
 'F9_03_PC_PROG_SVC_ACC_3_EXP',
 'F9_10_PC_ESCROW_LIABILITY_EOY',
 'F9_00_HD_FILER_CITY_US',
 'F9_09_PC_COMP_OFFICERS_PROG_SVCE',
 'F9_10_PC_LOANS_FROM_OFFICERS_EOY',
 'F9_03_PC_PROG_SVC_ACC_3_GRNT',
 'F9_09_PC_FEES_FOR_SVCE_OTH_TOT',
 'F9_09_PC_COMP_OFFICERS_TOTAL',
 'F9_09_EXP_INTEREST_TOT',

# Read 990 Data 
In the following code block we read the file (produced in the previous notebook) containing all filings into a PANDAS dataframe.

In [10]:
#%%time
#import datetime
#print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
#df = pd.read_pickle('all NEW filings February 2024 - all control variables (renamed).pkl.gz', compression='gzip')
#print('# of columns:', len(df.columns))
#print('# of observations:', len(df))
#df[:1]

Current date and time :  2024-03-29 23:59:41 

# of columns: 292
# of observations: 891980
CPU times: total: 1min 39s
Wall time: 1min 48s


,URL,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,BusinessName,BusinessNameControlTxt,PhoneNum,USAddress,InCareOfNm,ForeignAddress,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_06_PC_WHISTLEBLOWER_POLICY,F9_07_EXP_SCHED_O_X,F9_07_PC_COMPENSATION_OT

In [ ]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df = pd.read_feather('D:/all_filings_april_2025_all_controls_combined.feather')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
df[:1]

Current date and time :  2025-04-17 16:09:15 



In [17]:
df.dtypes

_id                                 object
OrganizationName                    object
URL                                 object
DLN                                 object
TaxPeriod                           object
                                     ...  
F9_12_PC_FED_GRNT_AUDIT_REQUIRED     int64
F9_12_PC_FINCL_STMTS_AUDITED         int64
F9_12_SCHED_O_X                      int64
number_of_other_prog_svces          object
501c3                                int32
Length: 300, dtype: object

In [18]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3469008 entries, 0 to 3469007
Data columns (total 300 columns):
 #    Column                              Dtype 
---   ------                              ----- 
 0    _id                                 object
 1    OrganizationName                    object
 2    URL                                 object
 3    DLN                                 object
 4    TaxPeriod                           object
 5    F9_09_PC_FEES_FOR_SVCE_FR_TOT       object
 6    F9_00_HD_BUILD_TIME_STAMP           object
 7    fiscal_year                         object
 8    EIN                                 object
 9    Name                                object
 10   NameControl                         object
 11   Phone                               object
 12   USAddress                           object
 13   ForeignAddress                      object
 14   InCareOfName                        object
 15   BusinessName                        object
 16 

In [14]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3469008 entries, 0 to 3469007
Data columns (total 300 columns):
 #    Column                              Dtype 
---   ------                              ----- 
 0    _id                                 object
 1    OrganizationName                    object
 2    URL                                 object
 3    DLN                                 object
 4    TaxPeriod                           object
 5    F9_09_PC_FEES_FOR_SVCE_FR_TOT       object
 6    F9_00_HD_BUILD_TIME_STAMP           object
 7    fiscal_year                         object
 8    EIN                                 object
 9    Name                                object
 10   NameControl                         object
 11   Phone                               object
 12   USAddress                           object
 13   ForeignAddress                      object
 14   InCareOfName                        object
 15   BusinessName                        object
 16 

In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
F9_00_HD_ADDR_CHANGE,3469008.0,0.039835,0.195571,0.0,0.0,0.0,0.0,1.0
F9_00_HD_AMENDED_RETURN,3469008.0,0.011920,0.108527,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_4847A1,3469008.0,0.000709,0.026626,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_501C3,3469008.0,0.752216,0.431726,0.0,1.0,1.0,1.0,1.0
F9_00_HD_FINAL_RETURN,3469008.0,0.005586,0.074533,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
F9_12_PC_FED_GRNT_AUDIT_PERFORMD,3469008.0,0.085901,0.280217,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FED_GRNT_AUDIT_REQUIRED,3469008.0,0.086664,0.281342,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FINCL_STMTS_AUDITED,3469008.0,0.434902,0.495744,0.0,0.0,0.0,1.0,1.0
F9_12_SCHED_O_X,3469008.0,0.165592,0.371714,0.0,0.0,0.0,0.0,1.0


In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
F9_00_HD_ADDR_CHANGE,3469008.0,0.039835,0.195571,0.0,0.0,0.0,0.0,1.0
F9_00_HD_AMENDED_RETURN,3469008.0,0.011920,0.108527,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_4847A1,3469008.0,0.000709,0.026626,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_501C3,3469008.0,0.752216,0.431726,0.0,1.0,1.0,1.0,1.0
F9_00_HD_FINAL_RETURN,3469008.0,0.005586,0.074533,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
F9_12_PC_FED_GRNT_AUDIT_PERFORMD,3469008.0,0.085901,0.280217,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FED_GRNT_AUDIT_REQUIRED,3469008.0,0.086664,0.281342,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FINCL_STMTS_AUDITED,3469008.0,0.434902,0.495744,0.0,0.0,0.0,1.0,1.0
F9_12_SCHED_O_X,3469008.0,0.165592,0.371714,0.0,0.0,0.0,0.0,1.0


In [10]:
%%time
import datetime
print("🕓 read started:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
# Step 3: Save
#df_clean.to_parquet("D:/filings_full.parquet", engine="pyarrow", compression="snappy", index=False)
df = pd.read_parquet("D:/all_filings_april_2025_all_controls_combined.parquet", engine="pyarrow")

print("✅ read completed:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("✅ Loaded:", df.shape)

🕓 read started: 2025-04-17 16:12:05
✅ read completed: 2025-04-17 16:15:26
✅ Loaded: (3469008, 300)
CPU times: total: 5min 43s
Wall time: 3min 20s


In [ ]:
#%%time
#import datetime
#print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
#df = pd.read_pickle('D:/all_filings_april_2025_all_controls_combined.pkl.gz', compression='gzip')
#print('# of columns:', len(df.columns))
#print('# of observations:', len(df))
#df[:1]

<br>Print out list of all 300 columns

In [20]:
print(df.columns.tolist())

['_id', 'OrganizationName', 'URL', 'DLN', 'TaxPeriod', 'F9_09_PC_FEES_FOR_SVCE_FR_TOT', 'F9_00_HD_BUILD_TIME_STAMP', 'fiscal_year', 'EIN', 'Name', 'NameControl', 'Phone', 'USAddress', 'ForeignAddress', 'InCareOfName', 'BusinessName', 'BusinessNameControlTxt', 'PhoneNum', 'InCareOfNm', 'ForeignPhoneNum', 'F9_00_HD_ADDR_CHANGE', 'F9_00_HD_AMENDED_RETURN', 'F9_00_HD_CTRY_OF_DOMICILE', 'F9_00_HD_EXEMPT_STATUS_4847A1', 'F9_00_HD_EXEMPT_STATUS_501C', 'F9_00_HD_EXEMPT_STATUS_501C3', 'F9_00_HD_FINAL_RETURN', 'F9_00_HD_GROSS_EXEMPT_NUM', 'F9_00_HD_GROSS_RCPT', 'F9_00_HD_GROUP_RETURN', 'F9_00_HD_INCLUDES_SUBORD_ORGS', 'F9_00_HD_INITIAL_RETURN', 'F9_00_HD_PRIN_OFF_NAME', 'F9_00_HD_SIGNING_OFFICER_SIGNTR', 'F9_00_HD_SPECIAL_CONDITION_DESC', 'F9_00_HD_STATE_OF_DOMICILE', 'F9_00_HD_TAX_PER_BEGIN', 'F9_00_HD_TAX_PER_END', 'F9_00_HD_TAX_YEAR', 'F9_00_HD_TIME_STAMP', 'F9_00_HD_TYPE_ORG_ASSOCIATION', 'F9_00_HD_TYPE_ORG_CORP', 'F9_00_HD_TYPE_ORG_OTHER', 'F9_00_HD_TYPE_ORG_OTHER_DESC', 'F9_00_HD_TYPE_ORG_

# CODE TO FLATTEN DICTIONARY

### Combine Variables in *Concordance* File

In [21]:
[c for c in df.columns if 'tax' in c.lower()]

['TaxPeriod',
 'F9_00_HD_TAX_PER_BEGIN',
 'F9_00_HD_TAX_PER_END',
 'F9_00_HD_TAX_YEAR',
 'F9_09_PC_PAYROLL_TAX_FUNDRAISE',
 'F9_09_PC_PAYROLL_TAX_MGMT',
 'F9_09_PC_PAYROLL_TAX_PROG_SVCE',
 'F9_09_PC_PAYROLL_TAX_TOTAL']

In [22]:
df[['F9_00_HD_BUILD_TIME_STAMP' ,'F9_00_HD_TIME_STAMP', 'TaxPeriod', 'F9_00_HD_TAX_YEAR', 'F9_00_HD_TAX_PER_END']].sample(5)

,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_TIME_STAMP,TaxPeriod,F9_00_HD_TAX_YEAR,F9_00_HD_TAX_PER_END
2912878,2023-04-26 12:10:37Z,2023-08-18T12:42:57-04:00,None,2022,2022-12-31
2675889,2022-09-23 18:48:47Z,2022-11-15T16:33:05-05:00,None,2021,2021-12-31
1072135,2017-02-10 21:41:12Z,2017-01-26T10:24:47-08:00,201606,2015,2016-06-30
689742,2016-02-25 16:41:14Z,2015-05-11T13:15:07-07:00,201412,2014,2014-12-31
1758216,2020-01-14 16:20:31Z,2019-11-12T13:04:07-06:00,201906,2018,2019-06-30


### Collapse concordance file
We'll aggregate the concordance file in order to get the list of valid 'sub-keys' for each nested/dictionary variable
- Note: I added 'cardinality' to *new_variables_df* in order to deal with *F9_09_EXP_OTH_TOT*

In [28]:
#def agg_funcs(x):
#    names = {
#        #'name': x['variable_name_new'].head(1).values[0],
#        'original_names':  list(set(x['MongoDB_Name'].tolist())),
#        'sub_keys':  list(set(x['sub_key'].tolist())),
#        'data_type_xsd': x['data_type_xsd'].head(1).values[0],
#        'cardinality': x['cardinality'].head(1).values[0]
#    }
#    #THE FOLLOWING SHORTCUT WORKS BUT CHANGES THE ORDER OF THE COLUMNS
#    #return pd.Series(names, index = list(names.keys()))
#    return pd.Series(names, index=['original_names', 'sub_keys', 'data_type_xsd', 'cardinality'])
#new_variables_df = concordance[concordance['sub_key'].notnull()][:].groupby(['variable_name_new']).apply(agg_funcs)
#new_variables_df = new_variables_df.reset_index()
#print('# of variables:', len(new_variables_df))
#new_variables_df[:]

#### New way
Here is the original code rewritten in the newer `.groupby().agg()` style for better readability and performance. This version handles all fields using named aggregation.

Notes:
- `list(set(x))` replaces the manual `set().tolist()` logic.
- `first` is used to pull the top value in the group for `data_type_xsd` and `cardinality`, just as `head(1).values[0]` did.
- I kept the `.reset_index()` for a flat DataFrame like your original.
- The filtering on `sub_key.notnull()` is preserved.



In [11]:
%%time
new_variables_df = (
    concordance[concordance['sub_key'].notnull()]
    .groupby('variable_name_new')
    .agg(
        original_names=('MongoDB_Name', lambda x: list(set(x))),
        sub_keys=('sub_key', lambda x: list(set(x))),
        data_type_xsd=('data_type_xsd', 'first'),
        cardinality=('cardinality', 'first')
    )
    .reset_index()
  )

print('# of variables:', len(new_variables_df))
new_variables_df

# of variables: 109
CPU times: total: 0 ns
Wall time: 17.6 ms


,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality
0,F9_00_HD_FILER_ADDR_US_L1,[Filer],[USAddress],StreetAddressType,None
1,F9_00_HD_FILER_ADDR_US_L2,[Filer],[USAddress],StreetAddressType,None
2,F9_00_HD_FILER_CITY_US,[Filer],[USAddress],CityType,None
3,F9_00_HD_FILER_COUNTRY_FRGN,[Filer],[ForeignAddress],CountryType,None
4,F9_00_HD_FILER_STATE_US,[Filer],[USAddress],StateType,None
...,...,...,...,...,...
104,F9_10_PC_SECURE_MORT_NOTES_EOY,"[MortNotesPyblSecuredInvestProp, MortgNotesPyblScrdInvstPropGrp]","[EOY, EOYAmt]",USAmountType,None
105,F9_10_PC_UNSECURED_LOANS_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOYAmt, EOY]",USAmountType,None
106,F9_10_PC_UNSECURED_NOTES_BOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[BOYAmt, BOY]",USAmountType,None
107,F9_10_PC_UNSECURED_NOTES_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOY, EOYAmt]",USAmountType,None


<br>One variable in this list has a value of 'MANY' for *cardinality*

In [30]:
new_variables_df['cardinality'].value_counts()

cardinality
ONE     36
MANY     1
Name: count, dtype: int64

In [31]:
new_variables_df[new_variables_df['cardinality']=='MANY']

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality
35,F9_09_EXP_OTH_TOT,"[OtherExpenses, OtherExpensesGrp]","[TotalAmt, Total]",USAmountType,MANY


<br>Create new variable in the collapsed concordance file to indicate the number of original names for each variable in the XML (e-file) data. Seven of the variables have one original name while 102 have two.

In [12]:
new_variables_df['len'] = new_variables_df['original_names'].apply(lambda x: len(x))
print(new_variables_df['len'].value_counts(), '\n')
new_variables_df

len
2    102
1      7
Name: count, dtype: int64 



,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len
0,F9_00_HD_FILER_ADDR_US_L1,[Filer],[USAddress],StreetAddressType,None,1
1,F9_00_HD_FILER_ADDR_US_L2,[Filer],[USAddress],StreetAddressType,None,1
2,F9_00_HD_FILER_CITY_US,[Filer],[USAddress],CityType,None,1
3,F9_00_HD_FILER_COUNTRY_FRGN,[Filer],[ForeignAddress],CountryType,None,1
4,F9_00_HD_FILER_STATE_US,[Filer],[USAddress],StateType,None,1
...,...,...,...,...,...,...
104,F9_10_PC_SECURE_MORT_NOTES_EOY,"[MortNotesPyblSecuredInvestProp, MortgNotesPyblScrdInvstPropGrp]","[EOY, EOYAmt]",USAmountType,None,2
105,F9_10_PC_UNSECURED_LOANS_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOYAmt, EOY]",USAmountType,None,2
106,F9_10_PC_UNSECURED_NOTES_BOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[BOYAmt, BOY]",USAmountType,None,2
107,F9_10_PC_UNSECURED_NOTES_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOY, EOYAmt]",USAmountType,None,2


<br>We'll also do the same for the number of sub-keys for each of the dictionary variables.

In [13]:
new_variables_df['len_subkeys'] = new_variables_df['sub_keys'].apply(lambda x: len(x))
print(new_variables_df['len_subkeys'].value_counts(), '\n')
new_variables_df

len_subkeys
2    100
1      9
Name: count, dtype: int64 



,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
0,F9_00_HD_FILER_ADDR_US_L1,[Filer],[USAddress],StreetAddressType,None,1,1
1,F9_00_HD_FILER_ADDR_US_L2,[Filer],[USAddress],StreetAddressType,None,1,1
2,F9_00_HD_FILER_CITY_US,[Filer],[USAddress],CityType,None,1,1
3,F9_00_HD_FILER_COUNTRY_FRGN,[Filer],[ForeignAddress],CountryType,None,1,1
4,F9_00_HD_FILER_STATE_US,[Filer],[USAddress],StateType,None,1,1
...,...,...,...,...,...,...,...
104,F9_10_PC_SECURE_MORT_NOTES_EOY,"[MortNotesPyblSecuredInvestProp, MortgNotesPyblScrdInvstPropGrp]","[EOY, EOYAmt]",USAmountType,None,2,2
105,F9_10_PC_UNSECURED_LOANS_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOYAmt, EOY]",USAmountType,None,2,2
106,F9_10_PC_UNSECURED_NOTES_BOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[BOYAmt, BOY]",USAmountType,None,2,2
107,F9_10_PC_UNSECURED_NOTES_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOY, EOYAmt]",USAmountType,None,2,2


### Write extended 'lambda' functions to parse sub-key variables
Here we will write two functions to deal with dictionary variables that have one and two nested sub-keys, respectively. I have leaned here on functions on Stack Overflow: https://stackoverflow.com/questions/48872234/using-apply-in-pandas-lambda-functions-with-multiple-if-statements?noredirect=1&lq=1

The trick is that these functions will return the nested sub-key value if it exists (and no value if the nested key(s) do not exist). We will apply these functions in loops later on in this notebook.

In [14]:
def func_onekey(x, key1):
    if pd.isnull(x):
        return np.nan
    #else: 
    #    mydict = ast.literal_eval(x)
    elif key1 in x.keys():
        return x[key1]
    else:
        return np.nan

In [15]:
def func(x, key1, key2):
    if pd.isnull(x):
        return np.nan
    #else: 
    #    mydict = ast.literal_eval(x)
    elif key1 in x.keys():
        return x[key1]
    elif key2 in x.keys():
        return x[key2]
    else:
        return np.nan

<br>Show the nine variables with a single nested sub-key

In [36]:
new_variables_df[new_variables_df['len_subkeys']!=2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
0,F9_00_HD_FILER_ADDR_US_L1,[Filer],[USAddress],StreetAddressType,None,1,1
1,F9_00_HD_FILER_ADDR_US_L2,[Filer],[USAddress],StreetAddressType,None,1,1
2,F9_00_HD_FILER_CITY_US,[Filer],[USAddress],CityType,None,1,1
3,F9_00_HD_FILER_COUNTRY_FRGN,[Filer],[ForeignAddress],CountryType,None,1,1
4,F9_00_HD_FILER_STATE_US,[Filer],[USAddress],StateType,None,1,1
5,F9_00_HD_FILER_ZIP_US,[Filer],[USAddress],ZIPCodeType,None,1,1
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[ProgSrvcAccomActy2Grp, Activity2]",[ActivityCode],IntegerNNType,None,2,1
12,F9_03_PC_PROG_SVC_ACC_3_CODE,"[Activity3, ProgSrvcAccomActy3Grp]",[ActivityCode],IntegerNNType,None,2,1
98,F9_10_PC_LOANS_FROM_OFFICERS_EOY,"[LoansFromOfficersDirectors, LoansFromOfficersDirectorsGrp]",[EOYAmt],USAmountType,None,2,1


### Process 
I first deal with a handful of variables a la carte. In future iterations of this notebook I will incorporate into the loop we'll be processing later on. 

Note: the reason these are being dealt with separately here is that the *Filer* variables contain double-nested data and I have yet to add the 'sub-sub-keys' to the concordance file. So, in the next code block I am first transforming the data for these four variables to be the value of the nested sub-key *USAddress*. Then in the subsequent code blocks we further transfrom these four variables to take the value of the sub-sub-key.

##### We won't do it this following way any more, because we've already parsed the *Filer* column

In [ ]:
#df[['F9_00_HD_FILER_ADDR_US_L1' ,'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_ZIP_US']].sample(5)

In [ ]:
#%%time
#df['F9_00_HD_FILER_ADDR_US_L1'] = df['Filer'][:].apply(func_onekey, key1='USAddress')
#df['F9_00_HD_FILER_ADDR_US_L2'] = df['Filer'][:].apply(func_onekey, key1='USAddress')
#df['F9_00_HD_FILER_CITY_US'] = df['Filer'][:].apply(func_onekey, key1='USAddress')
#df['F9_00_HD_FILER_ZIP_US'] = df['Filer'][:].apply(func_onekey, key1='USAddress')

In [37]:
df[['F9_09_PC_FEES_FOR_SVCE_FR_TOT', 'F9_00_HD_BUILD_TIME_STAMP', 
       'EIN', 'BusinessName', 'BusinessNameControlTxt', 'USAddress', 'PhoneNum', 'InCareOfNm',
       'ForeignAddress', 'ForeignPhoneNum']].sample(5)

,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,EIN,BusinessName,BusinessNameControlTxt,USAddress,PhoneNum,InCareOfNm,ForeignAddress,ForeignPhoneNum
1175070,"{""TotalAmt"": ""0""}",2017-02-10 21:41:12Z,330319261,{'BusinessNameLine1Txt': 'Corona Youth Tennis League'},CORO,"{'AddressLine1Txt': '2279 Eagle Glen Pkwy 112 PMB 356', 'CityNm': 'Corona', 'StateAbbreviationCd': 'CA', 'ZIPCd': '92883'}",9513714371,None,None,None
1010229,"{""TotalAmt"": ""0""}",2017-02-10 21:41:12Z,370922390,"{'BusinessNameLine1Txt': 'HUMAN RESOURCES CENTER OF EDGAR AND', 'BusinessNameLine2Txt': 'CLARK COUNTIES'}",HUMA,"{'AddressLine1Txt': 'P O BOX 1118', 'CityNm': 'PARIS', 'StateAbbreviationCd': 'IL', 'ZIPCd': '61944'}",2174654118,None,None,None
590870,None,2015-11-30 17:44:51Z,411841352,None,None,"{'AddressLine1': '579 WELLS STREET', 'City': 'ST PAUL', 'State': 'MN', 'ZIPCode': '55130'}",None,None,None,None
1721347,None,2019-02-21 02:37:17Z,510204577,"{'BusinessNameLine1Txt': ""EAST LOS ANGELES WOMEN'S CENTER""}",EAST,"{'AddressLine1Txt': '1431 S ATLANTIC BLVD', 'CityNm': 'LOS ANGELES', 'StateAbbreviationCd': 'CA', 'ZIPCd': '90022'}",3235265819,None,None,None
3289147,"{""TotalAmt"": ""0""}",2024-10-15 13:58:12Z,042104831,{'BusinessNameLine1Txt': 'PEOPLES INSTITUTE OF NORTHAMPTON INC'},PEOP,"{'AddressLine1Txt': '38 GOTHIC ST', 'CityNm': 'NORTHAMPTON', 'StateAbbreviationCd': 'MA', 'ZIPCd': '01060'}",4135848313,None,None,None


<br>The following variable is not of great interest to us so I'm just going to take the value of the entire *ForeignAddress* key and not parse it any further.

##### Updating approach for new XML filings
Actually, I'll do that later (see below)

In [33]:
#%%time
#df['F9_00_HD_FILER_COUNTRY_FRGN'] = df['Filer'][:].apply(func_onekey, key1='ForeignAddress')

Wall time: 9.05 s


In [38]:
print(len(df[df['ForeignAddress'].notnull()]))
df[df['ForeignAddress'].notnull()][['ForeignAddress']].sample(5)

3742


,ForeignAddress
2230376,"{'AddressLine1Txt': 'PO BOX 21345', 'CityNm': 'San Juan', 'ProvinceOrStateNm': 'Puerto Rico', 'CountryCd': 'RQ', 'ForeignPostalCd': '00928-1345'}"
2636540,"{'AddressLine1Txt': 'DEAN BRADLEY HOUSE 52 HORSEFERRY RD', 'CityNm': 'LONDON', 'CountryCd': 'UK', 'ForeignPostalCd': 'SW1P 2AF'}"
702803,"{'AddressLine1Txt': '272 ATTWELL DRIVE', 'CityNm': 'ETOBICOKE ONTARIO CA M', 'CountryCd': 'CA'}"
402855,"{'AddressLine1': 'KARLAVGEN 108', 'City': '5TH FLOOR', 'ProvinceOrState': 'STOCKHOLM', 'Country': 'SW', 'PostalCode': '11526'}"
2470884,"{'AddressLine1Txt': '31 33 Avenue Giuseppe Motta', 'CityNm': 'Geneva', 'ProvinceOrStateNm': 'Geneva', 'CountryCd': 'SZ', 'ForeignPostalCd': '1202'}"


In [32]:
#%%time
#df['F9_00_HD_FILER_COUNTRY_FRGN'] = df['ForeignAddress'][:].apply(func_onekey, key1='CountryCd')

CPU times: total: 31.2 ms
Wall time: 26.9 ms


In [33]:
#print(len(df[df['ForeignAddress'].notnull()]))
#df[df['ForeignAddress'].notnull()][['ForeignAddress', 'F9_00_HD_FILER_COUNTRY_FRGN']].sample(5)

30


,ForeignAddress,F9_00_HD_FILER_COUNTRY_FRGN
18059,"{'AddressLine1Txt': '502 ELLIS STREET', 'CityNm': 'PENTICTON', 'ProvinceOrStateNm': 'BC', 'CountryCd': 'CA', 'ForeignPostalCd': 'V2A 4M3'}",CA
13846,"{'AddressLine1Txt': 'WATERSIDESTRATFORD-UPON-AVON', 'CityNm': 'WARWICKSHIRE', 'CountryCd': 'UK', 'ForeignPostalCd': 'CV37 6BB'}",UK
894,"{'AddressLine1Txt': '2934 Baseline Road Suite 202', 'CityNm': 'OTTAWA', 'ProvinceOrStateNm': 'ONTARIO', 'CountryCd': 'CA', 'ForeignPostalCd': 'K2H 1B2'}",CA
19373,"{'AddressLine1Txt': 'PO BOX 10007 SUITE 104', 'CityNm': 'GUAYAMA', 'ProvinceOrStateNm': 'PUERTO RICO', 'CountryCd': 'RQ', 'ForeignPostalCd': '00785'}",RQ
4137,"{'AddressLine1Txt': 'PO Box 351', 'ProvinceOrStateNm': '1007, Lausanne', 'CountryCd': 'SZ'}",SZ


<br>Now let's parse the four variables noted above in order. Note that for each of these four we are applying our custom function ``func`` and transforming the variable to be the value of either of the two sub-keys. Recall a few things here. First, looking at *F9_00_HD_FILER_ADDR_US_L1*, above we have already changed this variable to be not all of what is contained under *Filer* but only the *USAddress* key. Below we then change the variable to be the value of either 'AddressLine1' or 'AddressLine1Txt'. Each filing will only have one of these two sub-keys depending on the year of the filing.

##### I've updated these four for the new XML filings

In [34]:
#%%time
#df['F9_00_HD_FILER_ADDR_US_L1'] = df['F9_00_HD_FILER_ADDR_US_L1'][:].apply(func, key1='AddressLine1', key2='AddressLine1Txt')

Wall time: 9.64 s


In [35]:
#%%time
#df['F9_00_HD_FILER_ADDR_US_L2'] = df['F9_00_HD_FILER_ADDR_US_L2'][:].apply(func, key1='AddressLine2', key2='AddressLine2Txt')

Wall time: 9.25 s


In [36]:
#%%time
#df['F9_00_HD_FILER_CITY_US'] = df['F9_00_HD_FILER_CITY_US'][:].apply(func, key1='City', key2='CityNm')

Wall time: 9.08 s


In [37]:
#%%time
#df['F9_00_HD_FILER_ZIP_US'] = df['F9_00_HD_FILER_ZIP_US'][:].apply(func, key1='ZIPCd', key2='ZIPCode')

Wall time: 9.07 s


In [39]:
df[['EIN', 'BusinessName', 'USAddress']].sample(5)

,EIN,BusinessName,USAddress
2380140,222488540,"{'BusinessNameLine1Txt': 'LAW ENFORCEMENT ALLIANCE OF SOUTH-', 'BusinessNameLine2Txt': 'INGTON'}","{'AddressLine1Txt': 'PO BOX 126', 'CityNm': 'SOUTHINGTON', 'StateAbbreviationCd': 'CT', 'ZIPCd': '06489'}"
2863386,840410760,{'BusinessNameLine1Txt': 'THE COLORADO MOUNTAIN CLUB'},"{'AddressLine1Txt': '710 10TH STREET SUITE 200', 'CityNm': 'GOLDEN', 'StateAbbreviationCd': 'CO', 'ZIPCd': '80401'}"
1862287,586066597,{'BusinessNameLine1Txt': 'SOUTHEASTERN CARP & MW HEALTH PLAN'},"{'AddressLine1Txt': 'PO BOX 1449', 'CityNm': 'GOODLETTSVILLE', 'StateAbbreviationCd': 'TN', 'ZIPCd': '370701449'}"
2892841,260030908,{'BusinessNameLine1Txt': 'THE DREAM PROGRAM INC'},"{'AddressLine1Txt': 'PO BOX 361', 'CityNm': 'WINOOSKI', 'StateAbbreviationCd': 'VT', 'ZIPCd': '05404'}"
447454,830246840,None,"{'AddressLine1': 'PO BOX 2799', 'City': 'CASPER', 'State': 'WY', 'ZIPCode': '82602'}"


#### Conversion code for feather/parquet files

In [16]:
import ast
import json

def convert_to_dict(x):
    """Convert string to dictionary if needed"""
    if pd.isnull(x):
        return np.nan
    
    if isinstance(x, dict):
        return x
        
    try:
        # Try using json.loads first (safer and handles more formats)
        return json.loads(x)
    except:
        try:
            # Fallback to ast.literal_eval
            return ast.literal_eval(x)
        except:
            return np.nan

In [ ]:
#%%time
## First convert strings to dictionaries
#df['F9_00_HD_EXEMPT_STATUS_501C'] = df['F9_00_HD_EXEMPT_STATUS_501C'].apply(convert_to_dict)

In [17]:
dict_cols = ['USAddress', 'ForeignAddress']
df[dict_cols].dtypes

USAddress         object
ForeignAddress    object
dtype: object

In [18]:
# Print a few non-null values to see what we're working with
sample_values = df[df[dict_cols[0]].notnull()][dict_cols[0]].head(3).tolist()
print(f"Sample values from {dict_cols[0]}:")
for val in sample_values:
    print(f"Value: {val}")
    print(f"Type: {type(val)}")

Sample values from USAddress:
Value: {'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300', 'City': 'BETHLEHEM', 'State': 'PA', 'ZIPCode': '18017'}
Type: <class 'str'>
Value: {'AddressLine1': '1660 DUKE STREET', 'City': 'ALEXANDRIA', 'State': 'VA', 'ZIPCode': '22314'}
Type: <class 'str'>
Value: {'AddressLine1': '1660 DUKE STREET', 'City': 'ALEXANDRIA', 'State': 'VA', 'ZIPCode': '22314'}
Type: <class 'str'>


In [19]:
%%time
# Apply the conversion to each column directly
for col in dict_cols:
    if col in df.columns:
        print(f"Converting {col}...")
        #df[col] = df[col].apply(convert_to_dict)
        df[col] = df[col].map(convert_to_dict)

Converting USAddress...
Converting ForeignAddress...
CPU times: total: 2min 6s
Wall time: 2min 8s


In [20]:
# Print a few non-null values to see what we're working with
sample_values = df[df[dict_cols[0]].notnull()][dict_cols[0]].head(3).tolist()
print(f"Sample values from {dict_cols[0]}:")
for val in sample_values:
    print(f"Value: {val}")
    print(f"Type: {type(val)}")

Sample values from USAddress:
Value: {'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300', 'City': 'BETHLEHEM', 'State': 'PA', 'ZIPCode': '18017'}
Type: <class 'dict'>
Value: {'AddressLine1': '1660 DUKE STREET', 'City': 'ALEXANDRIA', 'State': 'VA', 'ZIPCode': '22314'}
Type: <class 'dict'>
Value: {'AddressLine1': '1660 DUKE STREET', 'City': 'ALEXANDRIA', 'State': 'VA', 'ZIPCode': '22314'}
Type: <class 'dict'>


In [118]:
import gc
gc.collect()

513

#### Back to former code

Next time, consider this:


Step 1: Convert stringified dicts to actual dicts [if needed]
```python
df['USAddress'] = df['USAddress'].map(convert_to_dict)
```

Step 2: Extract value using a clean, efficient function
```python

def extract_first_available(d, *keys):
    if not isinstance(d, dict):
        return np.nan
    for key in keys:
        if key in d:
            return d[key]
    return np.nan

df['F9_00_HD_FILER_ZIP_US'] = df['USAddress'].map(lambda d: extract_first_available(d, 'ZIPCd', 'ZIPCode'))
```

#### New function

In [22]:
%%time
#df['F9_00_HD_FILER_ADDR_US_L1'] = df['USAddress'][:].apply(func, key1='AddressLine1', key2='AddressLine1Txt')
df['F9_00_HD_FILER_ADDR_US_L1'] = df['USAddress'].map(lambda d: extract_first_available(d, 'AddressLine1', 'AddressLine1Txt'))

CPU times: total: 13.8 s
Wall time: 15.1 s


In [25]:
%%time
#df['F9_00_HD_FILER_ADDR_US_L2'] = df['USAddress'][:].apply(func, key1='AddressLine2', key2='AddressLine2Txt')
df['F9_00_HD_FILER_ADDR_US_L2'] = df['USAddress'].map(lambda d: extract_first_available(d, 'AddressLine2', 'AddressLine2Txt'))

CPU times: total: 2.16 s
Wall time: 2.31 s


In [26]:
%%time
#df['F9_00_HD_FILER_CITY_US'] = df['USAddress'][:].apply(func, key1='City', key2='CityNm')
df['F9_00_HD_FILER_CITY_US'] = df['USAddress'].map(lambda d: extract_first_available(d, 'City', 'CityNm'))

CPU times: total: 1.88 s
Wall time: 2.12 s


In [27]:
%%time
#df['F9_00_HD_FILER_ZIP_US'] = df['USAddress'][:].apply(func, key1='ZIPCd', key2='ZIPCode')
df['F9_00_HD_FILER_ZIP_US'] = df['USAddress'].map(lambda d: extract_first_available(d, 'ZIPCd', 'ZIPCode'))

CPU times: total: 1.83 s
Wall time: 2.09 s


<br>Now let's take a look at these four variables in a random sample of five filings. All appear to be parsed successfully.

In [28]:
print(len(df[df['F9_00_HD_FILER_ADDR_US_L2'].notnull()]))

43179


In [29]:
%%time
df[['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_ZIP_US']].sample(5)

CPU times: total: 703 ms
Wall time: 775 ms


,F9_00_HD_FILER_ADDR_US_L1,F9_00_HD_FILER_ADDR_US_L2,F9_00_HD_FILER_CITY_US,F9_00_HD_FILER_ZIP_US
1996086,429 N MAIN STREET 2,NaN,GREENVILLE,29601
2598268,PO BOX 28,NaN,Bruceton Mills,26525
3097367,129 ELM STREET,NaN,VACAVILLE,95688
1572876,102 COURTHOUSE DRIVE,NaN,ST MATTHEWS,29135
2130624,700 Pass Road,NaN,Gulfport,39501


<br>Parse another variable and then show some descriptives.

##### Update variable for new XML filings

In [39]:
#%%time
#df['F9_00_HD_FILER_COUNTRY_FRGN'] = df['F9_00_HD_FILER_COUNTRY_FRGN'][:].apply(func, key1='Country', key2='CountryCd')

Wall time: 26.1 s


In [30]:
%%time
#df['F9_00_HD_FILER_COUNTRY_FRGN'] = df['ForeignAddress'][:].apply(func, key1='Country', key2='CountryCd')
df['F9_00_HD_FILER_COUNTRY_FRGN'] = df['ForeignAddress'].map(lambda d: extract_first_available(d, 'Country', 'CountryCd'))

CPU times: total: 1.64 s
Wall time: 1.72 s


In [60]:
df['F9_00_HD_FILER_COUNTRY_FRGN'].value_counts()

F9_00_HD_FILER_COUNTRY_FRGN
CA    1210
RQ     737
UK     382
IS     279
FR      82
      ... 
SP       1
ET       1
PS       1
TU       1
AL       1
Name: count, Length: 94, dtype: int64

In [31]:
%%time
df[['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_ZIP_US',
   'F9_00_HD_FILER_COUNTRY_FRGN']].describe().T

CPU times: total: 5.88 s
Wall time: 6.08 s


,count,unique,top,freq
F9_00_HD_FILER_ADDR_US_L1,3465266,636068,2335 NORTH BANK DRIVE,2260
F9_00_HD_FILER_ADDR_US_L2,43179,4612,Suite,13463
F9_00_HD_FILER_CITY_US,3465266,29086,NEW YORK,61732
F9_00_HD_FILER_ZIP_US,3465266,96995,20036,9132
F9_00_HD_FILER_COUNTRY_FRGN,3742,94,CA,1210


<br>Here I want to double-check that the parsing was correct for *F9_00_HD_FILER_COUNTRY_FRGN*. There are few observations in the dataset with a value for this variable, so rather than show a random sample of five observations, we take a random sample of five observations that actually have a value for this variable using the ``notnull()`` function.

In [32]:
%%time
df[df['F9_00_HD_FILER_COUNTRY_FRGN'].notnull()][['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_ZIP_US',
   'F9_00_HD_FILER_COUNTRY_FRGN']].sample(5)

CPU times: total: 250 ms
Wall time: 272 ms


,F9_00_HD_FILER_ADDR_US_L1,F9_00_HD_FILER_ADDR_US_L2,F9_00_HD_FILER_CITY_US,F9_00_HD_FILER_ZIP_US,F9_00_HD_FILER_COUNTRY_FRGN
2738345,NaN,NaN,NaN,NaN,UK
3089207,NaN,NaN,NaN,NaN,RQ
2809718,NaN,NaN,NaN,NaN,CA
2566885,NaN,NaN,NaN,NaN,CA
1497007,NaN,NaN,NaN,NaN,KR


In [42]:
#%%time
#df[['URL', 'F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US',
#    'F9_00_HD_FILER_ZIP_US', 'F9_00_HD_FILER_COUNTRY_FRGN']].to_pickle('efile address variables.pkl')

Wall time: 5.21 s


<br>Process variable for state. As with the four variables parsed above this is another 'two-step' parsing process.

##### Update for new XML filings

In [43]:
#%%time
#df['F9_00_HD_FILER_STATE_US'] = df['Filer'][:].apply(func_onekey, key1='USAddress')

Wall time: 9.41 s


In [ ]:
#%%time
#df['F9_00_HD_FILER_STATE_US'] = df['F9_00_HD_FILER_STATE_US'][:].apply(func, key1='State', key2='StateAbbreviationCd')

In [33]:
df[['EIN', 'USAddress']].sample(5)

,EIN,USAddress
2590898,391860172,"{'AddressLine1Txt': '16300 W NATIONAL AVENUE', 'CityNm': 'NEW BERLIN', 'StateAbbreviationCd': 'WI', 'ZIPCd': '53151'}"
604154,391563654,"{'AddressLine1': 'P O BOX 99', 'City': 'BLACK RIVER FALLS', 'State': 'WI', 'ZIPCode': '54615'}"
929665,541648579,"{'AddressLine1Txt': '5815 82ND STREET SUITE 145 317', 'CityNm': 'LUBBOCK', 'StateAbbreviationCd': 'TX', 'ZIPCd': '79424'}"
2781915,222146908,"{'AddressLine1Txt': '601 WASHINGTON AVENUE', 'CityNm': 'SOUTH AMBOY', 'StateAbbreviationCd': 'NJ', 'ZIPCd': '08879'}"
2548867,680027507,"{'AddressLine1Txt': '2577 CALIFORNIA PARK DRIVE', 'CityNm': 'CHICO', 'StateAbbreviationCd': 'CA', 'ZIPCd': '95928'}"


In [34]:
%%time
#df['F9_00_HD_FILER_STATE_US'] = df['USAddress'][:].apply(func, key1='State', key2='StateAbbreviationCd')
df['F9_00_HD_FILER_STATE_US'] = df['USAddress'].map(lambda d: extract_first_available(d, 'State', 'StateAbbreviationCd'))

CPU times: total: 2.28 s
Wall time: 2.44 s


In [65]:
#%%time
#print(len(df[df['F9_00_HD_FILER_STATE_US'].notnull()]))

3465266
CPU times: total: 18.8 s
Wall time: 19.2 s


#### This is much faster for counting non-nulls

In [35]:
%%time
df['F9_00_HD_FILER_STATE_US'].count()  # ✅ Fastest for non-nulls

CPU times: total: 312 ms
Wall time: 317 ms


3465266

In [36]:
%%time
df['F9_00_HD_FILER_STATE_US'].isna().sum()

CPU times: total: 281 ms
Wall time: 313 ms


3742

In [37]:
df[['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US',
                     'F9_00_HD_FILER_COUNTRY_FRGN', 'F9_00_HD_FILER_STATE_US', 'F9_00_HD_FILER_ZIP_US']].count()

F9_00_HD_FILER_ADDR_US_L1      3465266
F9_00_HD_FILER_ADDR_US_L2        43179
F9_00_HD_FILER_CITY_US         3465266
F9_00_HD_FILER_COUNTRY_FRGN       3742
F9_00_HD_FILER_STATE_US        3465266
F9_00_HD_FILER_ZIP_US          3465266
dtype: int64

<br>The code block above shows that there are 1,810 observations with a value for *Filer* that do not have a value for US state. To check what is going on here, I will run the following code block to show me the *Filer* column for a random sample of five observations that have a value for *Filer* but are missing the state variable. As you can see, the missing values all have a foreign address so we are in good shape.

In [46]:
#df[(df['Filer'].notnull())&(df['F9_00_HD_FILER_STATE_US'].isnull())][['Filer']].sample(5)

,Filer
950575,"{'EIN': '980506316', 'BusinessName': {'BusinessNameLine1Txt': 'GSM ASSOCIATION'}, 'InCareOfNm': '% OONAGH STEIN', 'BusinessNameControlTxt': 'GSMA', 'PhoneNum': '2073560600', 'ForeignAddress': {'AddressLine1Txt': 'FLOOR 2 WALBROOK BLDG 25 WALBROOK..."
159777,"{'EIN': '980160122', 'Name': {'BusinessNameLine1': 'Eshel-the Assn for the Planning & Development', 'BusinessNameLine2': 'of Services for the Aged in Israel'}, 'InCareOfName': '% ELIYAHU EREZ', 'NameControl': 'ESHE', 'Phone': '2126876200', 'Forei..."
420180,"{'EIN': '980437032', 'BusinessName': {'BusinessNameLine1': 'CANADIAN LUNG ASSOCIATION'}, 'BusinessNameControlTxt': 'CANA', 'PhoneNum': '6135696411', 'ForeignAddress': {'AddressLine1': '1750 Courtwood Crescent', 'AddressLine2': 'Suite 300', 'City'..."
1128020,"{'EIN': '391522897', 'BusinessName': {'BusinessNameLine1Txt': 'THE BRITISH NORTH AMERICA', 'BusinessNameLine2Txt': 'PHILATELIC SOCIETY LTD'}, 'BusinessNameControlTxt': 'BRIT', 'PhoneNum': '4104422040', 'ForeignAddress': {'AddressLine1Txt': '15 BR..."
1163810,"{'EIN': '981253233', 'BusinessName': {'BusinessNameLine1Txt': 'MUSEUM KAMPA - THE JAN AND MEDA', 'BusinessNameLine2Txt': 'MLADEK FOUNDATION'}, 'BusinessNameControlTxt': 'MUSE', 'PhoneNum': '3017188920', 'ForeignAddress': {'AddressLine1Txt': 'U SO..."


<br>Now show a random sample of the state variable for five observations.

In [38]:
df[['F9_00_HD_FILER_STATE_US']].sample(5)

,F9_00_HD_FILER_STATE_US
2777629,CA
2288069,NJ
2107366,OH
1050425,IN
952436,NY


<br>For further verification we can also check a sample of observations that have a value for *F9_00_HD_FILER_COUNTRY_FRGN*. All of the US variables are empty for these five observations so, again, we are in good shape.

In [39]:
df[df['F9_00_HD_FILER_COUNTRY_FRGN'].notnull()][['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_STATE_US',
    'F9_00_HD_FILER_ZIP_US', 'F9_00_HD_FILER_COUNTRY_FRGN']].sample(5)

,F9_00_HD_FILER_ADDR_US_L1,F9_00_HD_FILER_ADDR_US_L2,F9_00_HD_FILER_CITY_US,F9_00_HD_FILER_STATE_US,F9_00_HD_FILER_ZIP_US,F9_00_HD_FILER_COUNTRY_FRGN
1384129,NaN,NaN,NaN,NaN,NaN,CA
3356655,NaN,NaN,NaN,NaN,NaN,UK
2482536,NaN,NaN,NaN,NaN,NaN,UK
3408835,NaN,NaN,NaN,NaN,NaN,UK
257288,NaN,NaN,NaN,NaN,NaN,CA


In [70]:
df[['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_STATE_US',
    'F9_00_HD_FILER_ZIP_US', 'F9_00_HD_FILER_COUNTRY_FRGN']].sample(5)

,F9_00_HD_FILER_ADDR_US_L1,F9_00_HD_FILER_ADDR_US_L2,F9_00_HD_FILER_CITY_US,F9_00_HD_FILER_STATE_US,F9_00_HD_FILER_ZIP_US,F9_00_HD_FILER_COUNTRY_FRGN
3436058,1220 VALLEY FORGE ROAD SUITE 16,NaN,PHOENIXVILLE,PA,194602676,NaN
2569070,11561 STATE ROUTE 60,NaN,LOWELL,OH,45744,NaN
2497768,1010 N UNIVERSITY PARKS DR,NaN,WACO,TX,76707,NaN
1081701,PO BOX 12,NaN,LEXINGTON,MA,02420,NaN
240622,2002 LONDON ROAD,NaN,DULUTH,MN,55812,NaN


<br>Now let's drop *F9_00_HD_FILER_STATE_US* and other 'Filer' variables from *new_variables_df* because they are now dealt with above.

In [40]:
new_variables_df[:6]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
0,F9_00_HD_FILER_ADDR_US_L1,[Filer],[USAddress],StreetAddressType,None,1,1
1,F9_00_HD_FILER_ADDR_US_L2,[Filer],[USAddress],StreetAddressType,None,1,1
2,F9_00_HD_FILER_CITY_US,[Filer],[USAddress],CityType,None,1,1
3,F9_00_HD_FILER_COUNTRY_FRGN,[Filer],[ForeignAddress],CountryType,None,1,1
4,F9_00_HD_FILER_STATE_US,[Filer],[USAddress],StateType,None,1,1
5,F9_00_HD_FILER_ZIP_US,[Filer],[USAddress],ZIPCodeType,None,1,1


In [41]:
variables_to_drop = ['F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US',
                     'F9_00_HD_FILER_COUNTRY_FRGN', 'F9_00_HD_FILER_STATE_US', 'F9_00_HD_FILER_ZIP_US']
print(len(new_variables_df))
new_variables_df = new_variables_df[~new_variables_df['variable_name_new'].isin(variables_to_drop)]
print(len(new_variables_df))

109
103


In [42]:
new_variables_df[:2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
6,F9_00_HD_SIGNING_OFFICER_SIGNTR,"[BusinessOfficerGrp, Officer]","[SignatureDt, DateSigned]",DateType,None,2,2
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[Activity2, ProgSrvcAccomActy2Grp]",[ActivityCode],IntegerNNType,None,2,1


#### Also drop *Filer* from our PANDAS dataset

In [75]:
#df = df.drop('Filer', axis=1)

# 4/17/2025 - These three variables are missing!
The issue for `F9_03_PC_PROG_SVC_ACC_2_CODE` and `F9_03_PC_PROG_SVC_ACC_3_CODE` is that `ActivityCode` no longers seems to be included. Instead, it is things like {"Expense": "0", "Grants": "0", "Revenue": "0", "Description": "FINANCIAL COUNSELLING"}

In [130]:
#%%time
#import datetime
#print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
#dfx = pd.read_feather('D:/all_filings_april_2025_all_controls_combined.feather')
#print('# of columns:', len(dfx.columns))
#print('# of observations:', len(dfx))
#dfx[:1]

Current date and time :  2025-04-17 15:51:16 

# of columns: 300
# of observations: 3469008
CPU times: total: 4min 37s
Wall time: 4min


,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

In [137]:
#dfx[['F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE', 'F9_10_PC_LOANS_FROM_OFFICERS_EOY']].sample(5)

,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_10_PC_LOANS_FROM_OFFICERS_EOY
60461,None,None,None
1930237,None,None,None
2474388,None,None,None
3090841,None,None,None
1369567,None,None,None


In [45]:
import gc
gc.collect()

471

#### There needs to be `EOY` and `EOYAmt`!!!

In [51]:
df[df['F9_10_PC_LOANS_FROM_OFFICERS_EOY'].notnull()][[ 'F9_10_PC_LOANS_FROM_OFFICERS_EOY', #'F9_03_PC_PROG_SVC_ACC_1_CODE', 
              'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].sample(5)

,F9_10_PC_LOANS_FROM_OFFICERS_EOY,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE
2060184,"{""BOYAmt"": ""4150"", ""EOYAmt"": ""4407""}",None,None
1926716,"{""EOYAmt"": ""12000""}","{""Desc"": ""THE COALITION PROVIDED INFORMATION IN SPANISH TO INDIVIDUALS NEEDING IMMEDIATE ASSISTANCE AND ALLEVIATED CULTURAL AND LANGUAGE BARRIERS TO ACCESSING SERVICES.""}","{""Desc"": ""CREATED AND MAINTAINED REGIONAL ASSISTANCE NETWORKS OF BILINQUAL/BICULTURAL SERVICE PROVIDERS, COMMUNITY LEADERS, AND VOLUNTEERS TO DEVELOP STRATEGIES FOR ADDRESSING THE LOCAL NEEDS OF LATINO FAMILINES AND FILLING GAPS IN RESOURCES.""}"
1691999,"{""BOYAmt"": ""0"", ""EOYAmt"": ""0""}","{""ExpenseAmt"": ""2300549"", ""RevenueAmt"": ""422598"", ""Desc"": ""GIFFORD RETIREMENT COMMUNITY, INC., OPENED AN INDEPENDENT LIVING FACILITY IN AUGUST 2017. THIS FACILITY WILL AID THOSE 55 AND OVER LOOKING TO DOWNSIZE AND LIVE A MORE CAREFREE LIFESTYLE.""}","{""ExpenseAmt"": ""1271414"", ""RevenueAmt"": ""673456"", ""Desc"": ""GIFFORD RETIREMENT COMMUNITY, INC. OPERATES TWO ADULT DAY FACILITIES TO SERVE THE NEEDS OF THE ELDERLY AND DISABLED POPULATION IN THE AREA. THE FACILITIES ARE LOCATED IN BETHEL AND BARRE,..."
2958495,"{""BOYAmt"": ""0"", ""EOYAmt"": ""0""}","{""ExpenseAmt"": ""0"", ""GrantAmt"": ""0"", ""RevenueAmt"": ""0""}","{""ExpenseAmt"": ""0"", ""GrantAmt"": ""0"", ""RevenueAmt"": ""0""}"
1622574,"{""BOYAmt"": ""0"", ""EOYAmt"": ""0""}",None,None


### Loop over variables with a single sub-key
Now we'll proceed to a more efficient looping process. First, we will loop over the over the three variables that have a single sub-key and apply our 'one key' function to each of the three variables in turn.

In [52]:
new_variables_df[new_variables_df['len_subkeys']!=2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[Activity2, ProgSrvcAccomActy2Grp]",[ActivityCode],IntegerNNType,None,2,1
12,F9_03_PC_PROG_SVC_ACC_3_CODE,"[ProgSrvcAccomActy3Grp, Activity3]",[ActivityCode],IntegerNNType,None,2,1
98,F9_10_PC_LOANS_FROM_OFFICERS_EOY,"[LoansFromOfficersDirectorsGrp, LoansFromOfficersDirectors]",[EOYAmt],USAmountType,None,2,1


In [54]:
new_variables_df.at[
    new_variables_df[new_variables_df['variable_name_new'] == 'F9_10_PC_LOANS_FROM_OFFICERS_EOY'].index[0],
    'sub_keys'] = ['EOYAmt', 'EOY']

In [55]:
new_variables_df[new_variables_df['len_subkeys']!=2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[Activity2, ProgSrvcAccomActy2Grp]",[ActivityCode],IntegerNNType,None,2,1
12,F9_03_PC_PROG_SVC_ACC_3_CODE,"[ProgSrvcAccomActy3Grp, Activity3]",[ActivityCode],IntegerNNType,None,2,1
98,F9_10_PC_LOANS_FROM_OFFICERS_EOY,"[LoansFromOfficersDirectorsGrp, LoansFromOfficersDirectors]","[EOYAmt, EOY]",USAmountType,None,2,1


In [ ]:
#new_variables_df['len_subkeys'] = new_variables_df['sub_keys'].apply(len)

<br>In the following loop we will loop over each of the three above variables in *new_variables_df* and, taking the variable name and associated sub-key from *new_variables_df*, we will apply our custom ``func_onekey`` function to that variable in our e-filing dataset. 

In [129]:
#df[['LoansFromOfficersDirectors', 'LoansFromOfficersDirectorsGrp']][:2]

In [50]:
#%%time
#for index, row in new_variables_df[new_variables_df['len_subkeys']!=2].iterrows():
#    variable = row['variable_name_new']
#    keys = row['sub_keys']
#    key = keys[0]
#    #key2 = keys[1]
#    print(variable, key)
#    #print(type(row['variable_name_new']))
#    #df.loc[df.index[index], row['variable_name_new']] = 
#    #df.loc[df.index[45], 'reptrak100-rank-2013 (binary)'] = 0
#    
#    df[variable] = df[variable][:].apply(func_onekey, key1=key)

F9_03_PC_PROG_SVC_ACC_2_CODE ActivityCode
F9_03_PC_PROG_SVC_ACC_3_CODE ActivityCode
F9_10_PC_LOANS_FROM_OFFICERS_EOY EOYAmt
CPU times: total: 20.4 s
Wall time: 22.6 s


#### New way
✅ Why this is better:
Uses map() for speed and clarity.

Uses extract_first_available() to avoid rewriting logic.

No slicing ([:]) or use of index from iterrows() needed.

To extend to two keys:
```python
%%time
for _, row in new_variables_df.iterrows():
    variable = row['variable_name_new']
    subkeys = row['sub_keys']
    
    print(f"Processing: {variable} → {subkeys}")
    
    df[variable] = df[variable].map(lambda d: extract_first_available(d, *subkeys))
```

For example, the last row could be:
```python
df[variable] = df[variable].map(lambda d: extract_first_available(d, 'SignatureDt', 'DateSigned'))
```

In [74]:
new_variables_df[new_variables_df['len_subkeys'] != 2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[Activity2, ProgSrvcAccomActy2Grp]",[ActivityCode],IntegerNNType,None,2,1
12,F9_03_PC_PROG_SVC_ACC_3_CODE,"[ProgSrvcAccomActy3Grp, Activity3]",[ActivityCode],IntegerNNType,None,2,1
98,F9_10_PC_LOANS_FROM_OFFICERS_EOY,"[LoansFromOfficersDirectorsGrp, LoansFromOfficersDirectors]","[EOYAmt, EOY]",USAmountType,None,2,1


#### Convert to dict

In [76]:
for col in new_variables_df[new_variables_df['len_subkeys'] != 2]['variable_name_new']:
    if col in df.columns:
        print(f"Would convert: {col}")
    else:
        print(f"⚠️ Missing in df: {col}")

Would convert: F9_03_PC_PROG_SVC_ACC_2_CODE
Would convert: F9_03_PC_PROG_SVC_ACC_3_CODE
Would convert: F9_10_PC_LOANS_FROM_OFFICERS_EOY


In [78]:
%%time
# Apply the conversion to each column directly
for col in new_variables_df[new_variables_df['len_subkeys'] != 2]['variable_name_new']:
    if col in df.columns:
        print(f"Converting {col}...")
        #df[col] = df[col].apply(convert_to_dict)
        df[col] = df[col].map(convert_to_dict)

Converting F9_03_PC_PROG_SVC_ACC_2_CODE...
Converting F9_03_PC_PROG_SVC_ACC_3_CODE...
Converting F9_10_PC_LOANS_FROM_OFFICERS_EOY...
CPU times: total: 16.3 s
Wall time: 17.1 s


### Notes on new loop
🔧 TL;DR:
✅ Use for _, row in df.iterrows() when you don’t care about the index and want clean access to row values.

⚠️ Avoid for row in df.iterrows() unless you specifically want both index and Series together and plan to unpack them yourself.

In [79]:
%%time
for _, row in new_variables_df[new_variables_df['len_subkeys'] != 2].iterrows():
    variable = row['variable_name_new']
    subkey = row['sub_keys'][0]  # safe since len_subkeys == 1
    new_variable = f"{variable}_alt"  # new column name

    print(f"Processing: {variable} → {subkey} → storing in {new_variable}")
    
    df[new_variable] = df[variable].map(lambda d: extract_first_available(d, subkey))
    df[new_variable] = df[variable].map(lambda d: extract_first_available(d, *row['sub_keys']))

Processing: F9_03_PC_PROG_SVC_ACC_2_CODE → ActivityCode → storing in F9_03_PC_PROG_SVC_ACC_2_CODE_alt
Processing: F9_03_PC_PROG_SVC_ACC_3_CODE → ActivityCode → storing in F9_03_PC_PROG_SVC_ACC_3_CODE_alt
Processing: F9_10_PC_LOANS_FROM_OFFICERS_EOY → EOYAmt → storing in F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt
CPU times: total: 46.6 s
Wall time: 49.6 s


In [80]:
df[new_variables_df[new_variables_df['len_subkeys'] != 2]['variable_name_new'].tolist()].isna().sum()

F9_03_PC_PROG_SVC_ACC_2_CODE        2417803
F9_03_PC_PROG_SVC_ACC_3_CODE        2729652
F9_10_PC_LOANS_FROM_OFFICERS_EOY    3183768
dtype: int64

In [81]:
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt', 'F9_03_PC_PROG_SVC_ACC_2_CODE_alt', 'F9_03_PC_PROG_SVC_ACC_3_CODE_alt']].isna().sum()

F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt    3203031
F9_03_PC_PROG_SVC_ACC_2_CODE_alt        3468613
F9_03_PC_PROG_SVC_ACC_3_CODE_alt        3468726
dtype: int64

In [57]:
df[new_variables_df[new_variables_df['len_subkeys'] != 2]['variable_name_new'].tolist()].isna().sum()

F9_03_PC_PROG_SVC_ACC_2_CODE        2415853
F9_03_PC_PROG_SVC_ACC_3_CODE        2727658
F9_10_PC_LOANS_FROM_OFFICERS_EOY    3183767
dtype: int64

In [86]:
df[df['F9_10_PC_LOANS_FROM_OFFICERS_EOY'].notnull()][['F9_10_PC_LOANS_FROM_OFFICERS_EOY',
                                                      'F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt',
                                'F9_03_PC_PROG_SVC_ACC_2_CODE_alt', 'F9_03_PC_PROG_SVC_ACC_3_CODE_alt']].sample(15)

,F9_10_PC_LOANS_FROM_OFFICERS_EOY,F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt,F9_03_PC_PROG_SVC_ACC_2_CODE_alt,F9_03_PC_PROG_SVC_ACC_3_CODE_alt
1689360,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
2834420,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
1304733,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
220201,"{'BOY': '21069', 'EOY': '32574'}",32574,NaN,NaN
1923311,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
3050461,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
754205,{'EOYAmt': '134451'},134451,NaN,NaN
2275782,"{'BOYAmt': '764360', 'EOYAmt': '761589'}",761589,NaN,NaN
2852164,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN
2774044,"{'BOYAmt': '0', 'EOYAmt': '0'}",0,NaN,NaN


In [90]:
df[df['F9_03_PC_PROG_SVC_ACC_3_CODE_alt'].notnull()][['F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt', 'F9_03_PC_PROG_SVC_ACC_2_CODE',
                                 'F9_03_PC_PROG_SVC_ACC_2_CODE_alt', 'F9_03_PC_PROG_SVC_ACC_3_CODE',                     
                                 'F9_03_PC_PROG_SVC_ACC_3_CODE_alt']].sample(5)

,F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_CODE_alt,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE_alt
33778,NaN,"{'ActivityCode': '713990', 'Expense': '804832', 'Revenue': '1031532', 'Description': 'MEMBERSHIP DUES & ANUAC FEES: MEMBERSHIP DUES ARE ESTABLISHED & COLLECTED TO COVER NORMAL OPERATING EXPENSES OF INDIVIDUAL UNITS. MEMBERSHIP FEES INCLUDE MONTHL...",713990,"{'ActivityCode': '712110', 'Expense': '279871', 'Revenue': '246601', 'Description': 'MUSEUM ADMISSIONS FOR THE CAF ARE FOR MONIES COLLECTED BY INDIVIDUAL UNITS, LOCATED THROUGHOUT THE UNITED STATES, FOR ENTRY INTO THEIR FACILITIES. FACILITIES ARE...",712110
300660,NaN,"{'ActivityCode': '522130', 'Expense': '138473', 'Revenue': '934818', 'Description': 'INTEREST ON LOANS MADE TO MEMBERS LOAN FEES LOAN LOSSES'}",522130,"{'ActivityCode': '522130', 'Expense': '932492', 'Revenue': '66825', 'Description': 'ALL OTHER INCOME AND EXPENSES THAT SERVE BOTH PROGRAM SERVICES ABOVE INVESTMENT SALARIES BUILDING INSURANCE AUDITS ETC.'}",522130
191378,NaN,"{'ActivityCode': '90099', 'Expense': '13852', 'Description': 'WEDNESDAY NIGHT PROGRAM AND WORSHIP'}",90099,"{'ActivityCode': '90099', 'Expense': '6661', 'Description': 'OUTREACH PROGRAMS FOR COLLEGE STUDENTS'}",90099
64688,NaN,"{'ActivityCode': '2', 'Grants': '38000', 'Description': 'THE CONTINUOUS BLESSINGS FOUNDATION SPONSORED THE BOYS AND GIRLS CLUB OF WAKE COUNTY PROGRAM WHICH ALLOWS UNDERPRIVILEGED CHILDREN THE OPPORTUNITY TO PARTICIPATE IN AMATEUR ATHLETIC COMPETI...",2,"{'ActivityCode': '3', 'Grants': '2500', 'Description': 'THE CONTINUOUS BLESSINGS FOUNDATION SPONSORED THE FRIENDS OF THE WAKE COUNTY GUARDIAN AD LITEM BACK TO SCHOOL WITH PRIDE PROJECT.'}",3
374254,0,"{'ActivityCode': '611710', 'Expense': '1947705', 'Grants': '1930000', 'Description': 'Lightning Community Heroes - Lightning owner Jeff Vinik, along with his wife Penny, launched the Community Heroes initiative at the beginning of the 2011-2012 s...",611710,"{'ActivityCode': '611710', 'Expense': '63279', 'Revenue': '62638', 'Description': 'Hockey Camps - Lightning Made Hockey is designed to give hockey players or all ages and skill levels the opportunity to experience, learn, and enjoy the game of ho...",611710


#### Redo and overwrite

In [92]:
%%time
for _, row in new_variables_df[new_variables_df['len_subkeys'] != 2].iterrows():
    variable = row['variable_name_new']
    subkeys = row['sub_keys']  # may be 1 or more

    print(f"Processing: {variable} → {subkeys}")
    
    df[variable] = df[variable].map(lambda d: extract_first_available(d, *subkeys))

Processing: F9_03_PC_PROG_SVC_ACC_2_CODE → ['ActivityCode']
Processing: F9_03_PC_PROG_SVC_ACC_3_CODE → ['ActivityCode']
Processing: F9_10_PC_LOANS_FROM_OFFICERS_EOY → ['EOYAmt', 'EOY']
CPU times: total: 6.48 s
Wall time: 7.12 s


In [94]:
df[df['F9_10_PC_LOANS_FROM_OFFICERS_EOY'].notnull()][['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt', 'F9_03_PC_PROG_SVC_ACC_2_CODE',
                                 'F9_03_PC_PROG_SVC_ACC_2_CODE_alt', 'F9_03_PC_PROG_SVC_ACC_3_CODE',                     
                                 'F9_03_PC_PROG_SVC_ACC_3_CODE_alt']].sample(5)

,F9_10_PC_LOANS_FROM_OFFICERS_EOY,F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_CODE_alt,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE_alt
2270557,0,0,NaN,NaN,NaN,NaN
2247748,0,0,NaN,NaN,NaN,NaN
1165550,3714399,3714399,NaN,NaN,NaN,NaN
1156933,0,0,NaN,NaN,NaN,NaN
3029236,0,0,NaN,NaN,NaN,NaN


In [93]:
df[df['F9_03_PC_PROG_SVC_ACC_3_CODE'].notnull()][['F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt', 'F9_03_PC_PROG_SVC_ACC_2_CODE',
                                 'F9_03_PC_PROG_SVC_ACC_2_CODE_alt', 'F9_03_PC_PROG_SVC_ACC_3_CODE',                     
                                 'F9_03_PC_PROG_SVC_ACC_3_CODE_alt']].sample(5)

,F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_CODE_alt,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE_alt
120685,0,611110,611110,611110,611110
118594,0,561499,561499,561499,561499
625578,NaN,2,2,3,3
386410,NaN,61100,61100,531100,531100
86404,NaN,541900,541900,541900,541900


In [95]:
df = df.drop('F9_10_PC_LOANS_FROM_OFFICERS_EOY_alt', axis=1)
df = df.drop('F9_03_PC_PROG_SVC_ACC_2_CODE_alt', axis=1)
df = df.drop('F9_03_PC_PROG_SVC_ACC_3_CODE_alt', axis=1)

In [96]:
df[df['F9_03_PC_PROG_SVC_ACC_1_CODE'].notnull()][['F9_03_PC_PROG_SVC_ACC_1_CODE']].sample(5)

,F9_03_PC_PROG_SVC_ACC_1_CODE
915594,0
3453945,0
2568973,310000
1595885,162
1574738,611310


#### Save DF

In [97]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_feather('D:/all_filings_april_2025_all_controls_parsed.feather')

Current date and time :  2025-04-17 21:50:55 

CPU times: total: 2min 39s
Wall time: 2min 10s


In [ ]:
#%%time
#df.to_parquet("D:/all_filings_april_2025_all_controls_combined.parquet", engine="pyarrow", compression="snappy", index=False)

In [ ]:
#%%time
#import datetime
#print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
#df.to_pickle('D:/all_filings_april_2025_all_controls_combined.pkl.gz', compression='gzip')

<br>We can print out frequencies for the first two variables to verify that the above function worked correctly.

In [98]:
df['F9_03_PC_PROG_SVC_ACC_2_CODE'].value_counts()

F9_03_PC_PROG_SVC_ACC_2_CODE
2         49
522130    46
900099    34
611710    25
624100    21
          ..
221100     1
110000     1
488490     1
813410     1
711190     1
Name: count, Length: 88, dtype: int64

In [99]:
df['F9_03_PC_PROG_SVC_ACC_3_CODE'].value_counts()

F9_03_PC_PROG_SVC_ACC_3_CODE
3         35
522130    32
900099    31
611710    22
624100    17
          ..
100        1
10         1
71320      1
541100     1
623990     1
Name: count, Length: 64, dtype: int64

<br>Inspect the data type for the above three variables. All three are 'object' (text) type for now.

In [53]:
#%%time
#df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].dtypes

CPU times: total: 8.44 s
Wall time: 8.97 s


F9_10_PC_LOANS_FROM_OFFICERS_EOY     object
F9_03_PC_PROG_SVC_ACC_2_CODE        float64
F9_03_PC_PROG_SVC_ACC_3_CODE        float64
dtype: object

In [100]:
%%time
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].dtypes

CPU times: total: 93.8 ms
Wall time: 140 ms


F9_10_PC_LOANS_FROM_OFFICERS_EOY    object
F9_03_PC_PROG_SVC_ACC_2_CODE        object
F9_03_PC_PROG_SVC_ACC_3_CODE        object
dtype: object

<br>Look at a sample of 10 observations for these three variables.

In [107]:
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].count()

F9_10_PC_LOANS_FROM_OFFICERS_EOY    265977
F9_03_PC_PROG_SVC_ACC_2_CODE           395
F9_03_PC_PROG_SVC_ACC_3_CODE           282
dtype: int64

In [110]:
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].count()/len(df)

F9_10_PC_LOANS_FROM_OFFICERS_EOY    0.076672
F9_03_PC_PROG_SVC_ACC_2_CODE        0.000114
F9_03_PC_PROG_SVC_ACC_3_CODE        0.000081
dtype: float64

In [108]:
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].isna().sum()

F9_10_PC_LOANS_FROM_OFFICERS_EOY    3203031
F9_03_PC_PROG_SVC_ACC_2_CODE        3468613
F9_03_PC_PROG_SVC_ACC_3_CODE        3468726
dtype: int64

In [109]:
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].isna().sum()/len(df)

F9_10_PC_LOANS_FROM_OFFICERS_EOY    0.923328
F9_03_PC_PROG_SVC_ACC_2_CODE        0.999886
F9_03_PC_PROG_SVC_ACC_3_CODE        0.999919
dtype: float64

In [112]:
%%time
df[['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']].sample(15)

CPU times: total: 203 ms
Wall time: 269 ms


,F9_10_PC_LOANS_FROM_OFFICERS_EOY,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_3_CODE
635815,NaN,NaN,NaN
3208144,17257,NaN,NaN
2485154,NaN,NaN,NaN
2318258,NaN,NaN,NaN
116894,NaN,NaN,NaN
1973265,NaN,NaN,NaN
1180290,NaN,NaN,NaN
3421205,NaN,NaN,NaN
1723196,NaN,NaN,NaN
1725940,NaN,NaN,NaN


<br>Show the three variables in *new_variables_df*. We're using the ``isin()`` function to apply a filter (any rows where the value of *variable_name_new* matches the three in the list.

In [113]:
new_variables_df[new_variables_df['variable_name_new'].isin(['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE'])]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
7,F9_03_PC_PROG_SVC_ACC_2_CODE,"[Activity2, ProgSrvcAccomActy2Grp]",[ActivityCode],IntegerNNType,None,2,1
12,F9_03_PC_PROG_SVC_ACC_3_CODE,"[ProgSrvcAccomActy3Grp, Activity3]",[ActivityCode],IntegerNNType,None,2,1
98,F9_10_PC_LOANS_FROM_OFFICERS_EOY,"[LoansFromOfficersDirectorsGrp, LoansFromOfficersDirectors]","[EOYAmt, EOY]",USAmountType,None,2,1


<br>Drop *F9_10_PC_LOANS_FROM_OFFICERS_EOY*, *F9_03_PC_PROG_SVC_ACC_2_CODE*, and *F9_03_PC_PROG_SVC_ACC_3_CODE* from ``new_variables_df``. They are already dealt with above.

In [114]:
variables_to_drop = ['F9_10_PC_LOANS_FROM_OFFICERS_EOY', 'F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']
print(len(new_variables_df))
new_variables_df = new_variables_df[~new_variables_df['variable_name_new'].isin(variables_to_drop)]
print(len(new_variables_df))

103
100


#### Loop and apply main function
All of the remaining variables in *new_variables_df* have two sub-keys, as verified by the empty data outputted by the following line.

In [115]:
new_variables_df[new_variables_df['len_subkeys']!=2]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys


<br>I'm pasting in our custom function ``func`` here again to have it handy.

In [116]:
def func(x, key1, key2):
    if pd.isnull(x):
        return np.nan
    #else: 
    #    mydict = ast.literal_eval(x)
    elif key1 in x.keys():
        return x[key1]
    elif key2 in x.keys():
        return x[key2]
    else:
        return np.nan

<br>We can see that there are 100 variables with two sub-keys. We will process all 100 in one loop.

In [117]:
print(len(new_variables_df[new_variables_df['len_subkeys']!=2]))
print(len(new_variables_df[(new_variables_df['len_subkeys']==2)]))

0
100


In [119]:
new_variables_df[(new_variables_df['len_subkeys']==2)]

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys
6,F9_00_HD_SIGNING_OFFICER_SIGNTR,"[BusinessOfficerGrp, Officer]","[SignatureDt, DateSigned]",DateType,None,2,2
8,F9_03_PC_PROG_SVC_ACC_2_DESC,"[Activity2, ProgSrvcAccomActy2Grp]","[Desc, Description]",ExplanationType,None,2,2
9,F9_03_PC_PROG_SVC_ACC_2_EXP,"[Activity2, ProgSrvcAccomActy2Grp]","[Expense, ExpenseAmt]",USAmountType,None,2,2
10,F9_03_PC_PROG_SVC_ACC_2_GRNT,"[Activity2, ProgSrvcAccomActy2Grp]","[GrantAmt, Grants]",USAmountType,None,2,2
11,F9_03_PC_PROG_SVC_ACC_2_REV,"[Activity2, ProgSrvcAccomActy2Grp]","[RevenueAmt, Revenue]",USAmountType,None,2,2
...,...,...,...,...,...,...,...
104,F9_10_PC_SECURE_MORT_NOTES_EOY,"[MortNotesPyblSecuredInvestProp, MortgNotesPyblScrdInvstPropGrp]","[EOY, EOYAmt]",USAmountType,None,2,2
105,F9_10_PC_UNSECURED_LOANS_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOYAmt, EOY]",USAmountType,None,2,2
106,F9_10_PC_UNSECURED_NOTES_BOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[BOYAmt, BOY]",USAmountType,None,2,2
107,F9_10_PC_UNSECURED_NOTES_EOY,"[UnsecuredNotesLoansPayableGrp, UnsecuredNotesLoansPayable]","[EOY, EOYAmt]",USAmountType,None,2,2


### Main Loop
Now we will loop over all 100 variables that have two subkeys. For each variable we will apply our custom function ``func``. If you're wondering why there are two keys, recall that each variable, for example ``F9_09_PC_COMP_OFFICERS_TOTAL ``, was the result of combining two different XML sections from the e-file data. Not only were there different variable names, but the nested variable names were distinct. For this example variable there are two different variable 'keys'' -- ``Total`` and ``TotalAmt`` so we will take the values from either of these two keys. 

Accordingly, below you will see for each variable a printout of the variable name and the two keys from which we are grabbing data. 

In [120]:
df[['F9_09_EXP_OTH_TOT']].sample(5)

,F9_09_EXP_OTH_TOT
2511900,"[{""Desc"": ""COMMISSIONING AND DECOMMISIONING DOCKS"", ""TotalAmt"": ""8350"", ""ProgramServicesAmt"": ""8350""}, {""Desc"": ""BANK AND CREDIT CARD PROCESSING FEES"", ""TotalAmt"": ""1597"", ""ProgramServicesAmt"": ""1597""}, {""Desc"": ""GENERAL REPAIRS AND MAINTENANCE"",..."
595117,"[{""Desc"": ""POSTAGE & PRINTING"", ""TotalAmt"": ""1507027"", ""ProgramServicesAmt"": ""1449101"", ""ManagementAndGeneralAmt"": ""57926""}, {""Desc"": ""CREDIT CARD/BANK FEES"", ""TotalAmt"": ""676160"", ""ProgramServicesAmt"": ""660249"", ""ManagementAndGeneralAmt"": ""15911..."
2950950,None
2425050,"[{""Desc"": ""CAMP MAINTENANCE"", ""TotalAmt"": ""15424"", ""ProgramServicesAmt"": ""15424"", ""ManagementAndGeneralAmt"": ""0"", ""FundraisingAmt"": ""0""}, {""Desc"": ""MISCELLANEOUS"", ""TotalAmt"": ""9207"", ""ProgramServicesAmt"": ""95"", ""ManagementAndGeneralAmt"": ""8887"",..."
156309,"[{""Description"": ""BANQUET"", ""Total"": ""2686"", ""ProgramServices"": ""2686""}, {""Description"": ""POSTAGE"", ""Total"": ""2465"", ""ProgramServices"": ""2465""}, {""Description"": ""ALUMNI SHIRTS"", ""Total"": ""761"", ""ProgramServices"": ""761""}, {""Description"": ""LICENSE ..."


In [98]:
#%%time
#for index, row in new_variables_df[new_variables_df['len_subkeys']==2][:].iterrows():
#    variable = row['variable_name_new']
#    keys = row['sub_keys']
#    key1 = keys[0]
#    key2 = keys[1]
#    print(variable, key1, key2)
#    #print(type(row['variable_name_new']))
#    #df.loc[df.index[index], row['variable_name_new']] = 
#    #df.loc[df.index[45], 'reptrak100-rank-2013 (binary)'] = 0
#    
#    df[variable] = df[variable][:].apply(func, key1=key1, key2=key2)
#    #df[variable] = df[variable].astype('float')

<br>Note that the above will cause an error when we get to the variable *F9_09_EXP_OTH_TOT*. The reason, as you can see below, is that this is the only variable with a *cardinality* value of 'MANY'. So, before running the code, you may want to simply delete *F9_09_EXP_OTH_TOT* from *new_variables_df* before proceeding. 

In case you're interested, the error you would see is the following: ``ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()``

In [127]:
new_variables_df[new_variables_df['variable_name_new']=='F9_09_EXP_OTH_TOT']

,variable_name_new,original_names,sub_keys,data_type_xsd,cardinality,len,len_subkeys


#### The Fix: Drop *F9_09_EXP_OTH_TOT* from *new_variables_df* and *df*

In [126]:
variables_to_drop = ['F9_09_EXP_OTH_TOT']
print(len(new_variables_df))
new_variables_df = new_variables_df[~new_variables_df['variable_name_new'].isin(variables_to_drop)]
print(len(new_variables_df))

100
99


<br>Alternatively, you can skip this variable by running our loop twice. First, modify the second row to stop looping at the 28th element:

``for index, row in new_variables_df[new_variables_df['len_subkeys']==2][:27].iterrows():``

and then run it again startign at the 29th element:


``for index, row in new_variables_df[new_variables_df['len_subkeys']==2][28:].iterrows():``

The easiest solution, though, is to simply delete *F9_09_EXP_OTH_TOT* from *new_variables_df* and *df* and then run the main loop above. There are few research cases where you might want this variable.

<br>Here we can see a sample of five rows for *F9_09_EXP_OTH_TOT* and *F9_09_EXP_OTH_OTH_TOT*.

In [70]:
df[['F9_09_EXP_OTH_TOT', 'F9_09_EXP_OTH_OTH_TOT']].sample(5)

,F9_09_EXP_OTH_TOT,F9_09_EXP_OTH_OTH_TOT
406557,"[{'Desc': 'ASSOCIATION MEMBERSHIPS', 'TotalAmt': '16037', 'ProgramServicesAmt': '10000', 'ManagementAndGeneralAmt': '6037'}, {'Desc': 'PAYMENT PROCESSING FEES', 'TotalAmt': '10485', 'ProgramServicesAmt': '10485'}, {'Desc': 'BANK CHARGES', 'TotalA...","{'TotalAmt': '613', 'ManagementAndGeneralAmt': '613'}"
30053,"[{'Desc': 'TELEPHONE', 'TotalAmt': '7157', 'ProgramServicesAmt': '1638', 'ManagementAndGeneralAmt': '3965', 'FundraisingAmt': '1554'}, {'Desc': 'TRAINING', 'TotalAmt': '2887', 'ProgramServicesAmt': '1010', 'ManagementAndGeneralAmt': '1684', 'Fund...","{'TotalAmt': '396', 'ProgramServicesAmt': '324', 'ManagementAndGeneralAmt': '5', 'FundraisingAmt': '67'}"
92290,"[{'Desc': 'MAINTENANCE CONTRACT', 'TotalAmt': '361255', 'ProgramServicesAmt': '361255'}, {'Desc': 'SUPPLIES', 'TotalAmt': '13973', 'ProgramServicesAmt': '13973'}, {'Desc': 'EQUIPMENT RENTAL', 'TotalAmt': '1002', 'ProgramServicesAmt': '1002'}]",NaN
496251,"[{'Desc': 'SYMPOSIUM EXPENSES', 'TotalAmt': '99172', 'ProgramServicesAmt': '99172'}, {'Desc': 'RECONCILIATION PARK MAI', 'TotalAmt': '98017', 'ProgramServicesAmt': '98017'}, {'Desc': 'FACILITIES AND EQUIPMEN', 'TotalAmt': '13971', 'ManagementAndG...","{'TotalAmt': '1956', 'ManagementAndGeneralAmt': '1956'}"
262039,"[{'Desc': 'CONSULTING', 'TotalAmt': '12000', 'ProgramServicesAmt': '12000'}, {'Desc': 'PROFESSIONAL SERVICES', 'TotalAmt': '2722', 'ProgramServicesAmt': '2722'}, {'Desc': 'FEES', 'TotalAmt': '1107', 'ProgramServicesAmt': '1107'}, {'Desc': 'ENTERT...","{'TotalAmt': '98', 'ProgramServicesAmt': '98'}"


<br>Let's drop *F9_09_EXP_OTH_TOT* from our PANDAS dataset.

In [129]:
df = df.drop('F9_09_EXP_OTH_TOT', axis=1)

<br>Create a list of all 99 (now) sub-key variables.

In [130]:
subkey_vars = new_variables_df[new_variables_df['sub_keys'].notnull()]['variable_name_new'].tolist()
print(len(subkey_vars))

99


#### Convert to dict

In [132]:
%%time
print(len(subkey_vars))
# Print a few non-null values to see what we're working with
sample_values = df[df[subkey_vars[0]].notnull()][subkey_vars[0]].head(3).tolist()
print(f"Sample values from {subkey_vars[0]}:")
for val in sample_values:
    print(f"Value: {val}")
    print(f"Type: {type(val)}")

99
Sample values from F9_00_HD_SIGNING_OFFICER_SIGNTR:
Value: {'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}
Type: <class 'str'>
Value: {'Name': 'THOMAS D TURNBULL', 'Title': 'ASST. SEC/TREAS', 'DateSigned': '2011-11-09'}
Type: <class 'str'>
Value: {'Name': 'THOMAS D TURNBULL', 'Title': 'ASST. SEC/TREAS', 'DateSigned': '2011-11-09'}
Type: <class 'str'>
CPU times: total: 40 s
Wall time: 40.8 s


In [138]:
import gc
gc.collect()

0

#### Save DF

In [137]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_feather('D:/all_filings_april_2025_all_controls_parsed.feather')

Current date and time :  2025-04-17 22:48:38 

CPU times: total: 6min 14s
Wall time: 7min 10s


In [134]:
%%time
# Apply the conversion to each column directly
for col in subkey_vars:
    if col in df.columns:
        print(f"Converting {col}...")
        #df[col] = df[col].apply(convert_to_dict)
        df[col] = df[col].map(convert_to_dict)

Converting F9_00_HD_SIGNING_OFFICER_SIGNTR...
Converting F9_03_PC_PROG_SVC_ACC_2_DESC...
Converting F9_03_PC_PROG_SVC_ACC_2_EXP...
Converting F9_03_PC_PROG_SVC_ACC_2_GRNT...
Converting F9_03_PC_PROG_SVC_ACC_2_REV...
Converting F9_03_PC_PROG_SVC_ACC_3_DESC...
Converting F9_03_PC_PROG_SVC_ACC_3_EXP...
Converting F9_03_PC_PROG_SVC_ACC_3_GRNT...
Converting F9_03_PC_PROG_SVC_ACC_3_REV...
Converting F9_08_PC_TOTAL_REVENUE...
Converting F9_09_EXP_AD_PROMO_TOT...
Converting F9_09_EXP_BENF_PAID_MEMB_TOT...
Converting F9_09_EXP_CONF_MEETING_TOT...
Converting F9_09_EXP_DEPREC_FUNDR...
Converting F9_09_EXP_DEPREC_MAG...
Converting F9_09_EXP_DEPREC_PROG...
Converting F9_09_EXP_DEPREC_TOT...
Converting F9_09_EXP_GRANT_FRGN_TOT...
Converting F9_09_EXP_GRANT_INDIV_DMSTC_TOT...
Converting F9_09_EXP_GRANT_ORG_DMSTC_TOT...
Converting F9_09_EXP_INFO_TECH_TOT...
Converting F9_09_EXP_INSURANCE_TOT...
Converting F9_09_EXP_INTEREST_TOT...
Converting F9_09_EXP_JOINT_COSTS_TOT...
Converting F9_09_EXP_OCCUPANCY_

In [135]:
%%time
print(len(subkey_vars))
# Print a few non-null values to see what we're working with
sample_values = df[df[subkey_vars[0]].notnull()][subkey_vars[0]].head(3).tolist()
print(f"Sample values from {subkey_vars[0]}:")
for val in sample_values:
    print(f"Value: {val}")
    print(f"Type: {type(val)}")

99
Sample values from F9_00_HD_SIGNING_OFFICER_SIGNTR:
Value: {'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}
Type: <class 'dict'>
Value: {'Name': 'THOMAS D TURNBULL', 'Title': 'ASST. SEC/TREAS', 'DateSigned': '2011-11-09'}
Type: <class 'dict'>
Value: {'Name': 'THOMAS D TURNBULL', 'Title': 'ASST. SEC/TREAS', 'DateSigned': '2011-11-09'}
Type: <class 'dict'>
CPU times: total: 1min 25s
Wall time: 2min 36s


In [139]:
import gc
gc.collect()

0

##### Re-run loop
`4/17/2025 - Use new loop below this one`

In [73]:
#%%time
#for index, row in new_variables_df[new_variables_df['len_subkeys']==2][:].iterrows():
#    variable = row['variable_name_new']
#    keys = row['sub_keys']
#    key1 = keys[0]
#    key2 = keys[1]
#    print(variable, key1, key2)
#    #print(type(row['variable_name_new']))
#    #df.loc[df.index[index], row['variable_name_new']] = 
#    #df.loc[df.index[45], 'reptrak100-rank-2013 (binary)'] = 0
#    
#    df[variable] = df[variable][:].apply(func, key1=key1, key2=key2)
#    #df[variable] = df[variable].astype('float')

F9_00_HD_SIGNING_OFFICER_SIGNTR SignatureDt DateSigned
F9_03_PC_PROG_SVC_ACC_2_DESC Description Desc
F9_03_PC_PROG_SVC_ACC_2_EXP Expense ExpenseAmt
F9_03_PC_PROG_SVC_ACC_2_GRNT GrantAmt Grants
F9_03_PC_PROG_SVC_ACC_2_REV RevenueAmt Revenue
F9_03_PC_PROG_SVC_ACC_3_DESC Description Desc
F9_03_PC_PROG_SVC_ACC_3_EXP Expense ExpenseAmt
F9_03_PC_PROG_SVC_ACC_3_GRNT GrantAmt Grants
F9_03_PC_PROG_SVC_ACC_3_REV RevenueAmt Revenue
F9_08_PC_TOTAL_REVENUE TotalRevenueColumnAmt TotalRevenueColumn
F9_09_EXP_AD_PROMO_TOT TotalAmt Total
F9_09_EXP_BENF_PAID_MEMB_TOT TotalAmt Total
F9_09_EXP_CONF_MEETING_TOT TotalAmt Total
F9_09_EXP_DEPREC_FUNDR FundraisingAmt Fundraising
F9_09_EXP_DEPREC_MAG ManagementAndGeneral ManagementAndGeneralAmt
F9_09_EXP_DEPREC_PROG ProgramServicesAmt ProgramServices
F9_09_EXP_DEPREC_TOT TotalAmt Total
F9_09_EXP_GRANT_FRGN_TOT TotalAmt Total
F9_09_EXP_GRANT_INDIV_DMSTC_TOT TotalAmt Total
F9_09_EXP_GRANT_ORG_DMSTC_TOT TotalAmt Total
F9_09_EXP_INFO_TECH_TOT TotalAmt Total
F9_09_E

In [140]:
%%time
for _, row in new_variables_df[new_variables_df['len_subkeys'] == 2].iterrows():
    variable = row['variable_name_new']
    subkeys = row['sub_keys']  # will contain exactly two keys

    print(f"Processing: {variable} → {subkeys}")
    
    df[variable] = df[variable].map(lambda d: extract_first_available(d, *subkeys))

Processing: F9_00_HD_SIGNING_OFFICER_SIGNTR → ['SignatureDt', 'DateSigned']
Processing: F9_03_PC_PROG_SVC_ACC_2_DESC → ['Desc', 'Description']
Processing: F9_03_PC_PROG_SVC_ACC_2_EXP → ['Expense', 'ExpenseAmt']
Processing: F9_03_PC_PROG_SVC_ACC_2_GRNT → ['GrantAmt', 'Grants']
Processing: F9_03_PC_PROG_SVC_ACC_2_REV → ['RevenueAmt', 'Revenue']
Processing: F9_03_PC_PROG_SVC_ACC_3_DESC → ['Desc', 'Description']
Processing: F9_03_PC_PROG_SVC_ACC_3_EXP → ['Expense', 'ExpenseAmt']
Processing: F9_03_PC_PROG_SVC_ACC_3_GRNT → ['GrantAmt', 'Grants']
Processing: F9_03_PC_PROG_SVC_ACC_3_REV → ['RevenueAmt', 'Revenue']
Processing: F9_08_PC_TOTAL_REVENUE → ['TotalRevenueColumn', 'TotalRevenueColumnAmt']
Processing: F9_09_EXP_AD_PROMO_TOT → ['TotalAmt', 'Total']
Processing: F9_09_EXP_BENF_PAID_MEMB_TOT → ['TotalAmt', 'Total']
Processing: F9_09_EXP_CONF_MEETING_TOT → ['Total', 'TotalAmt']
Processing: F9_09_EXP_DEPREC_FUNDR → ['FundraisingAmt', 'Fundraising']
Processing: F9_09_EXP_DEPREC_MAG → ['Manage

<br>Show a sample of five rows for each of the 99 variables. As you can see none is still 'nested'. Our variable transformations conducted in our main loop above have worked. 

In [141]:
gc.collect()

0

In [142]:
%%time
print(len(df.columns), len(df))
print(len(df[subkey_vars].columns), len(df))
df[subkey_vars].sample(10)

305 3469008
99 3469008
CPU times: total: 26.3 s
Wall time: 27.3 s


,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_08_PC_TOTAL_REVENUE,F9_09_EXP_AD_PROMO_TOT,F9_09_EXP_BENF_PAID_MEMB_TOT,F9_09_EXP_CONF_MEETING_TOT,F9_09_EXP_DEPREC_FUNDR,F9_09_EXP_DEPREC_MAG,F9_09_EXP_DEPREC_PROG,F9_09_EXP_DEPREC_TOT,F9_09_EXP_GRANT_FRGN_TOT,F9_09_EXP_GRANT_INDIV_DMSTC_TOT,F9_09_EXP_GRANT_ORG_DMSTC_TOT,F9_09_EXP_INFO_TECH_TOT,F9_09_EXP_INSURANCE_TOT,F9_09_EXP_INTEREST_TOT,F9_09_EXP_JOINT_COSTS_TOT,F9_09_EXP_OCCUPANCY_TOT,F9_09_EXP_OFFICE_TOT,F9_09_EXP_OTH_OTH_TOT,F9_09_EXP_ROY_TOT,F9_09_EXP_TRAVEL_ENTRTNMNT_TOT,F9_09_EXP_TRAVEL_TOT,F9_09_PC_COMP_DISQUAL_FUNDRAISE,F9_09_PC_COMP_DISQUAL_MGMT,F9_09_PC_COMP_DISQUAL_PROG_SVCE,F9_09_PC_COMP_DISQUAL_TOTAL,F9_09_PC_COMP_OFFICERS_FUNDRAISE,F9_09_PC_COMP_OFFICERS_MGMT,F9_09_PC_COMP_OFFICERS_PROG_SVCE,F9_09_PC_COMP_OFFICERS_TOTAL,F9_09_PC_FEES_FOR_SVCE_ACCT_TOT,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_09_PC_FEES_FOR_SVCE_INVST_TOT,F9_09_PC_FEES_FOR_SVCE_LEGL_TOT,F9_09_PC_FEES_FOR_SVCE_LOBB_TOT,F9_09_PC_FEES_FOR_SVCE_MGMT_TOT,F9_09_PC_FEES_FOR_SVCE_OTH_TOT,F9_09_PC_OTHER_EMP_BEN_FUNDRAISE,F9_09_PC_OTHER_EMP_BEN_MGMT,F9_09_PC_OTHER_EMP_BEN_PROG_SVCE,F9_09_PC_OTHER_EMP_BEN_TOTAL,F9_09_PC_OTHER_SALARY_FUNDRAISE,F9_09_PC_OTHER_SALARY_MGMT,F9_09_PC_OTHER_SALARY_PROG_SVCE,F9_09_PC_OTHER_SALARY_TOTAL,F9_09_PC_PAYMENT_TO_AFFILIATES,F9_09_PC_PAYROLL_TAX_FUNDRAISE,F9_09_PC_PAYROLL_TAX_MGMT,F9_09_PC_PAYROLL_TAX_PROG_SVCE,F9_09_PC_PAYROLL_TAX_TOTAL,F9_09_PC_PENSION_CONT_FUNDRAISE,F9_09_PC_PENSION_CONT_MGMT,F9_09_PC_PENSION_CONT_PROG_SVCE,F9_09_PC_PENSION_CONT_TOTAL,F9_09_PC_TOTAL_FUNC_EXPENSES,F9_09_PC_TOTAL_FUNDRAISE_EXPENSE,F9_09_PC_TOTAL_MGMT_EXPENSE,F9_09_PC_TOTAL_PROG_SVCE_EXPENSE,F9_10_ASSETS_ACC_NET_EOY,F9_10_ASSETS_EXP_PREPAID_EOY,F9_10_ASSETS_INTANGIB_EOY,F9_10_ASSETS_INVENT_SALE_EOY,F9_10_ASSETS_LESS_DEPREC_EOY,F9_10_ASSETS_LOANS_DISQUAL_EOY,F9_10_ASSETS_NOTES_LOANS_NET_EOY,F9_10_ASSETS_OTH_EOY,F9_10_ASSETS_PLEDGES_NET_EOY,F9_10_LIAB_ACC_PAYABLE_EOY,F9_10_LIAB_GRANTS_PAYABLE_EOY,F9_10_LIAB_LOANS_OFF_EOY,F9_10_LIAB_REV_DEFERRED_EOY,F9_10_NAFB_RESTRICT_PERM_EOY,F9_10_NAFB_RESTRICT_TEMP_EOY,F9_10_NAFB_UNRESTRICT_EOY,F9_10_PC_BOND_LIABILITY_EOY,F9_10_PC_CASH_NON_INTEREST_BOY,F9_10_PC_CASH_NON_INTEREST_EOY,F9_10_PC_ESCROW_LIABILITY_EOY,F9_10_PC_INVEST_OTHER_SEC_EOY,F9_10_PC_INVEST_PROG_RELTD_EOY,F9_10_PC_INVEST_PUB_TRADED_EOY,F9_10_PC_OTHER_LIABILITIES_EOY,F9_10_PC_RET_EARNINGS_ENDWMT_EOY,F9_10_PC_SAVINGS_TEMP_INVEST_BOY,F9_10_PC_SAVINGS_TEMP_INVEST_EOY,F9_10_PC_SECURED_MORTGAGES_EOY,F9_10_PC_SECURE_MORT_NOTES_EOY,F9_10_PC_UNSECURED_LOANS_EOY,F9_10_PC_UNSECURED_NOTES_BOY,F9_10_PC_UNSECURED_NOTES_EOY,F9_10_PZ_TOTAL_ASSETS_EOY
1865558,2020-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2062335,13746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2468,17343,59862,NaN,NaN,184402,24976,70834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79807,79807,159614,15700,NaN,NaN,NaN,NaN,NaN,25102,NaN,43068,35562,78630,NaN,342160,574261,916421,NaN,NaN,27910,57214,85124,NaN,24787,714,25501,2171712,0,643009,1528703,55733,17537,NaN,35461,NaN,NaN,NaN,0,NaN,130841,NaN,NaN,0,NaN,NaN,-134968,NaN,102518,135698,NaN,NaN,NaN,NaN,289540,NaN,40958,40984,NaN,NaN,NaN,NaN,NaN,285413
1523879,2018-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1537298,3389,0,0,NaN,NaN,6488,6488,686233,0,0,0,6469,6375,NaN,0,0,74468,0,0,13010,NaN,NaN,NaN,0,17500,NaN,17500,35000,0,0,0,0,0,0,33245,NaN,NaN,NaN,0,28132,166448,15098,209678,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,1168787,150086,282350,736351,0,0,0,0,383327,0,0,0,0,17714,NaN,NaN,NaN,NaN,83432,582383,NaN,74447,404094,NaN,0,0,0,NaN,NaN,NaN,0,103892,103892,NaN,NaN,NaN,787421
1250364,2017-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1028711,NaN,NaN,3473,NaN,2583,49086,51669,NaN,NaN,NaN,NaN,32349,11344,NaN,67987,9852,12014,NaN,NaN,1561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8300,NaN,NaN,NaN,NaN,NaN,5769,NaN,1963,63670,65633,1000,46219,569343

### Look at 501(c)(3)s

In [143]:
print('# of columns:', len(df.columns))
print('# of observations:', len(df))

# of columns: 305
# of observations: 3469008


In [144]:
df['501c3'].value_counts()

501c3
1    2637781
0     831227
Name: count, dtype: int64

In [145]:
print(len(df[df['501c3']==1]))

2637781


#### Create and save list of EINs for BMF File

In [147]:
ein_list_2025_new = df[df['501c3']==1]['EIN'].tolist()
print(len(ein_list_2025_new))
print(len(set(ein_list_2025_new)))
ein_list_2025_new = list(set(ein_list_2025_new))
print(len(ein_list_2025_new))

2637781
353116
353116


In [148]:
gc.collect()

476

In [149]:
import json
with open('ein_list_501c3_new_2025.json', 'w') as fp:
    json.dump(ein_list_2025_new, fp)

### Check for which variables might be missing

In [150]:
new_cols = concordance[:]['variable_name_new'].tolist()
print(len(new_cols))
print(len(set(new_cols)))
new_cols = list(set(new_cols))
print(len(new_cols))
print(new_cols[:5])

574
288
288
['F9_04_PC_PROF_FR_EXP_GT_15K', 'F9_00_HD_EXEMPT_STATUS_501C', 'F9_01_PC_TOT_UBI_NET', 'F9_09_PC_TOTAL_FUNC_EXPENSES', 'F9_00_HD_TYPE_ORG_CORP']


<br>As shown in the next code block, only one variables is missing, which we deleted.

In [152]:
set(new_cols) - set(df.columns.tolist())

{'F9_09_EXP_OTH_TOT'}

In [153]:
set(df.columns.tolist()) - set(new_cols)

{'501c3',
 'BusinessName',
 'BusinessNameControlTxt',
 'DLN',
 'EIN',
 'ForeignAddress',
 'ForeignPhoneNum',
 'InCareOfName',
 'InCareOfNm',
 'Name',
 'NameControl',
 'OrganizationName',
 'Phone',
 'PhoneNum',
 'URL',
 'USAddress',
 '_id',
 'fiscal_year'}

#### Save DF

In [155]:
len(df)

3469008

In [ ]:
def prepare_for_save(df):
    import gc

    # Drop any cached views
    df = df.copy()  # Break reference to any partial evaluation from .head(), etc.

    # Optionally sort or reset if needed
    # df = df.sort_values("some_column")  # Only if relevant
    # df = df.reset_index(drop=True)

    # Trigger garbage collection
    gc.collect()

    print("🧼 DataFrame copied + garbage collected. Ready to save.")
    return df

In [ ]:
# Step 2: Clean up df (especially if you’ve been doing .head(), .sort(), etc.)
df = prepare_for_save(df)

In [156]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_feather('D:/all_filings_april_2025_all_controls_combined_parsed.feather')

Current date and time :  2025-04-18 00:30:15 

CPU times: total: 1min 43s
Wall time: 1min 34s


In [157]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_parquet("D:/all_filings_april_2025_all_controls_combined_parsed.parquet", engine="pyarrow", compression="snappy", index=False)

Current date and time :  2025-04-18 00:37:47 

CPU times: total: 2min 31s
Wall time: 2min 36s


In [158]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_pickle('D:/all_filings_april_2025_all_controls_combined_parsed.pkl.gz', compression='gzip')

Current date and time :  2025-04-18 00:41:12 

CPU times: total: 30min 11s
Wall time: 30min 47s
